# Capstone Project - A Restaurant In Boston
<i>Applied Data Science Capstone, IBM/Coursera Data Science Professional Certification</i>

## Table of contents
* [Introduction](#introduction)
* [Data/Exploration](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

# <span style="color:darkred">Introduction</span> <a name="introduction"></a>

For my Capstone Project I'll be examining the question of where to put a restaurant in Boston, Massachusetts. The city of Boston has a rich culinary history, covering the gamut from New England seafood ("say chowdah!") to the Italian restaurants and bakeries of the North End to Chinatown to the ubiquitous Dunkin' Donuts, based in nearby Canton MA. With roughly 1700 restaurant venues (including 74 Dunkin' Donuts), there's no shortage of options. So where might one squeeze in another venue?

# <span style="color:darkred">Data/Exploration</span> <a name="data"></a>

I didn't initially have any specific agenda in terms of how I'd solve the problem, so I pulled data from a host of different sources, demographics, venues, etc.; mapped it, sliced and diced it, looked for something to jump out, and eventually I found an angle. Before we get to that in the Methodology section, let's take a look at the data sources used...

<h3>Venue info</h3>

Foursquare EXPLORE endpoint
<br>https://api.foursquare.com/v2/venues/explore

<h3>Census Tract and Population info</h3>

US Census Bureau geocoder, using benchmark 4 (Public_AR_Current) and vintage 410 (Census2010_Current)
<br>https://geocoding.geo.census.gov/geocoder/

<h3>Household Income info</h3>

US Census Bureau American Community Survey 5-Year Data (2009-2019) (aka ACS5), report B19013_001E (Estimate Median household income in the past 12 months in 2019 inflation-adjusted dollars)
<br>https://api.census.gov/data/2019/acs/acs5

<h3>Boston Census Tracts</h3>

Analyze Boston
<br>https://data.boston.gov/dataset/census-2010-tracts1 via hub.arcgis.com

<h3>Boston Census Tract to Neighborhood key</h3>

Boston Planning & Development Agency; Neighborhood-Map-by-Census-Tract_Updated-March-2014-(1)-(2).pdf
<br>http://www.bostonplans.org/ 

<h3>MBTA Rapid Transit, Commuter Rail, and Bus info</h3>

Massachusetts Bureau of Geographic Information (MassGIS)
<br>https://docs.digital.mass.gov/dataset/massgis-data-mbta-rapid-transit via hub.arcgis.com
<br>https://docs.digital.mass.gov/dataset/massgis-data-trains via hub.arcgis.com
<br>https://docs.digital.mass.gov/dataset/massgis-data-mbta-bus-routes-and-stops via hub.arcgis.com

# <span style="color:darkred">Methodology</span> <a name="methodology"></a>

I started this project with a clean sheet and no preconceived notions about the direction I'd go. I could focus on a particular restaurant type, perhaps look for an under-represented cuisine (although that could raise questions on whether a given category is under-represented and an opportunity, or under-represented because the market has already spoken regarding that type of restaurant). Going a different direction, I could focus on a particular part of town which seemed ripe for growth.

One decision I made early on was that as part of the project, I'll be breaking up the city into neighborhoods as we did in the NYC and Toronto exercises, but with a twist. In those exercises we used a list of points as centroids of city neighborhoods, and defined venues as being "in" those neighborhoods based on being within a certain radius of the centroid. I understand that this was done for the sake of simplicity and keeping us focused on the task at hand, but in reality a neighborhood is not typically a point or defined simply by proximity to a point. It is a bounded area, usually irregularly shaped. For this project I'll be dividing the map of Boston into real neighborhoods and placing the venues pulled from Foursquare into these irregularly shaped areas.

The City of Boston Planning and Development Agency maintains all manner of maps and charts and so on regarding the defined neighborhoods of the city. Most critically for our purposes, they also maintain a "decoder ring" of what U.S. Census tracts correspond to what neighborhoods. Combining the city info with the census info will allow us to create our geographic plots.

### Datawork Begins: Installs and Imports

In [ ]:
# imports
import descartes                                        # working with geometric objects, dependency of geopandas
import fiona                                            # reading/writing GIS data, dependency of geopandas
import folium                                           # map rendering
from folium import plugins                              # folium plugins
from folium.features import DivIcon                     # making fast markers
from folium.plugins import HeatMap                      # making heatmaps
from geojson import Point, MultiPolygon, Feature        # geography tools
import geopandas as gpd                                 # working with geospatial data
from geopy.geocoders import Nominatim                   # finding lat/long from address
from io import StringIO                                 # creating in-memory file-like objects
import itertools                                        # creating iterators for efficient looping
import json                                             # handling JSON files
import math                                             # mathematical functions defined by the C standard
import matplotlib                                       # creating static, animated, and interactive visualizations
import matplotlib.cm as cm                              # colormap handling
import matplotlib.colors as colors                      # converting numbers or color arguments to RGB/A
import matplotlib.image as mpimg                        # basic image loading/rescaling/isplay
import matplotlib.lines as mlines                       # line editing
import matplotlib.pyplot as plt                         # interactive plots
import numpy as np                                      # multi-dimensional arrays and high-level mathematical functions
import os                                               # using operating system dependent functionality
import pandas as pd                                     # data analysis and manipulation
from pandas import json_normalize                       # transforming JSON file into a pandas dataframe
import plotly
import plotly.figure_factory as ff
import plotly.express as px
import pyproj                                           # map projections, dependency of geopandas
from pyproj import CRS                                  # coordinate reference system info
from pyproj import Proj                                 # coordinate transformation between CRS types
from pyproj import Transformer                          # making repeated transformations faster
from pywaffle import Waffle                             # waffle charts
import requests                                         # handling requests such as when webscraping
import rtree                                            # advanced spatial indexing features, dependency of geopandas
from scipy.spatial.distance import cdist                # computing distance between points using Euclidean distance
import shapefile                                        # working with shapefiles
import shapely                                          # working with geo shapes, dependency of geopandas
from shapely import geometry                            # shapely module
from shapely.geometry import shape, Point, Polygon      # shapely module
from shapely.geometry.multipolygon import MultiPolygon  # shapely module
from shapely.ops import unary_union, transform          # shapely module
import sklearn                                          # scikit-learn predictive data analysis
from sklearn.cluster import KMeans                      # module for KMeans Clustering
from textwrap import fill                               # wrap-around text in legends etc
import turfpy                                           # polygon/map tools
from turfpy import measurement                          # map measurements
from turfpy.measurement import boolean_point_in_polygon # determining if a point is within a polygon
import urllib.request                                   # pulling in URL data

print('...LIBRARIES IMPORTED')

In [ ]:
# disable false "chained assignment" warnings
pd.options.mode.chained_assignment = None  # default='warn'
print('...WARNING DISABLED')

### Setting Basic Variables

In [ ]:
# hidden variables
my_id = 'XXXXX'
my_secret = 'XXXXX'
my_token = 'XXXXX'
census_key = 'XXXXX'
MBTA_key = 'XXXXX'
print('...HIDDEN VARIABLES SET')

In [ ]:
# setting some basic variables
CLIENT_ID = my_id                          # your Foursquare ID
CLIENT_SECRET = my_secret                  # your Foursquare Secret
ACCESS_TOKEN = my_token                    # your Foursquare Access Token
VERSION = '20210501'                       # Foursquare API version
LIMIT = 1000                               # A default Foursquare API limit value
RADIUS = 2000                              # search / explore radius in meters
VENUE_SEARCH = '4af3a181f964a520fcee21e3'  # ID of venue to be searched for
SEARCH_LOC = 'Boston, MA'                  # location key for SEARCH or EXPLORE endpoint (near=)
SEARCH_CAT_ID = '4d4b7105d754a06374d81259' # category key for SEARCH or EXPLORE endpoint (categoryId=)
SECTION = 'food'                           # section key for EXPLORE endpoint (section=)
OFFSET1 = 100                              # offset key for EXPLORE endpoint (offset=)
OFFSET2 = 200                              # offset key for EXPLORE endpoint (offset=)
OFFSET3 = 300                              # offset key for EXPLORE endpoint (offset=)
print('...FOURSQUARE API CREDENTIALS SET')

In [ ]:
# geolocator to get basic coordinates
address = 'Boston, MA'
geolocator = Nominatim(user_agent="rsgd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Boston coordinates: latitude",latitude,"/ longitude",longitude)

In [ ]:
# workaround for issues with local GEOS install
shapely.speedups.disable()
print("...SHAPELY SPEEDUPS DISABLED")

### File Imports

In [ ]:
# imports
# create shortened url for github files
datafile_url = ("https://raw.githubusercontent.com/rsgdodge/Coursera_Capstone/master/data")

# MBTA Commuter Rail Stations
MBTA_Rail_Stop=gpd.read_file('https://opendata.arcgis.com/datasets/b741e3542a9f40898d49e776452efe63_0.geojson')
# MBTA Commuter Rail Lines
MBTA_Rail_Line=gpd.read_file('https://opendata.arcgis.com/datasets/b741e3542a9f40898d49e776452efe63_3.geojson')
# MBTA Rapid Transit Stops
MBTA_RT_Stop=gpd.read_file('https://opendata.arcgis.com/datasets/a9e4d01cbfae407fbf5afe67c5382fde_0.geojson')
# MBTA Rapid Transit Lines
MBTA_RT_Line=gpd.read_file('https://opendata.arcgis.com/datasets/a9e4d01cbfae407fbf5afe67c5382fde_3.geojson')
# MBTA Bus Routes
MBTA_Bus_Line=gpd.read_file('https://opendata.arcgis.com/datasets/cef8d0fe8b9d49fe9aa8f1b7524c6ac2_3.geojson')
# download Boston Census Tracts geojson
BosTract=gpd.read_file('https://opendata.arcgis.com/datasets/4a8eb4fb3be44ed5a1eec28551b9f3b2_0.geojson')

# download github csv files as DataFrames; neighborhood to census key and FourSquare broad categories
BPDA_Neighborhoods = pd.read_csv(f'{datafile_url}/BPDA_Neighborhoods.csv')
Broad_Categories = pd.read_csv(f'{datafile_url}/Broad_Categories.csv')
problem_hex = pd.read_csv(f'{datafile_url}/problem_hex.csv').astype('object')

print("...JSON FILES IMPORTED")

I took the census tract map of Boston from ArcGis and used GeoPandas "dissolve" to remove some of the internal boundaries of the geo data and aggregate the 180 census tracts into the defined neighborhoods, as well as dissolving all internal features to create a single city-sized object, we'll need that later.

In [ ]:
# working with imported JSONs
# removing large water-only tract
BosTractTemp = pd.DataFrame(BosTract, copy=True)
BosTractTemp.drop(BosTractTemp[BosTractTemp['ALAND10'] < 1].index, inplace = True)
BostonNoWater = gpd.GeoDataFrame(BosTractTemp, crs=4326)
# dissolving interior borders to make an "all of Boston" geo
BostonOutline = BostonNoWater.dissolve(aggfunc = 'sum')                                   # dissolving boundaries
BostonOutline.drop(columns=['FID', 'OBJECTID'], inplace=True)                             # dropping columns
BostonOutline32619 = BostonOutline.to_crs(epsg=32619)                                     # outline in alt CRS (m)
# dissolving interior borders to make a "Boston Neighborhoods" geo
BosHoods = pd.merge(BostonNoWater, BPDA_Neighborhoods, how='left', on='NAMELSAD10')
BosHoods.drop(columns=['GEOID10_y', 'NAME10_y'], inplace=True)                            # drop duplicate columns
BosHoods.rename(columns={"2014 BPDA Neighborhood":"Neighborhood","GEOID10_x":"GEOID10",   # rename columns
                           "NAME10_x":"NAME10"}, inplace=True)                            # rename columns  
BostonHoods = BosHoods.dissolve(by='Neighborhood', aggfunc = 'sum')                       # dissolving boundaries
BostonHoods.drop(columns=['FID', 'OBJECTID'], inplace=True)                               # dropping columns
BostonHoods.reset_index(inplace=True, drop=False)                                         # putting index back in
print("BosTract",'Type:',type(BosTract),"/ current CRS is:",BosTract.crs,"/ shape:",BosTract.shape)
print("BostonNoWater",'Type:',type(BostonNoWater),"/ current CRS is:",BostonNoWater.crs,"/ shape:",BostonNoWater.shape)
print("BostonOutline",'Type:',type(BostonOutline),"/ current CRS is:",BostonOutline.crs,"/ shape:",BostonOutline.shape)
print("BostonOutline32619",'Type:',type(BostonOutline32619),"/ current CRS is:",BostonOutline32619.crs,"/ shape:",BostonOutline32619.shape)
print("BostonHoods",'Type:', type(BostonHoods), "/ current CRS is:",BostonHoods.crs,"/ shape:",BostonHoods.shape)

In [ ]:
# finding the centroid of the BostonOutline polygon
# centroid in EPSG:4326 throws error, need to re-projecting from other
BostonCentroid32619 = BostonOutline32619.geometry.centroid         # pulling centroid location
BostonCentroid = BostonCentroid32619.to_crs(epsg=4326)             # converting from UTM style point back to lat/lon
print(" BostonCentroid (4326): ",BostonCentroid,"\n BostonCentroid32619: ",BostonCentroid32619)

In [ ]:
BostonCentroid

In [ ]:
# finding coordinate min/max via Shapely .bounds
Bos_minX = BostonOutline.bounds.minx # -71.191156
Bos_minY = BostonOutline.bounds.miny # 42.227889
Bos_maxX = BostonOutline.bounds.maxx # -70.920102
Bos_maxY = BostonOutline.bounds.maxy # 42.404937
Bos_minX32619 = BostonOutline32619.bounds.minx # 319335.919566
Bos_minY32619 = BostonOutline32619.bounds.miny # 4.677277e+06 int is 4677277
Bos_maxX32619 = BostonOutline32619.bounds.maxx # 341792.806255
Bos_maxY32619 = BostonOutline32619.bounds.maxy # 4.696683e+06 int is 4696683
# setting center points via centroid
Bos_centerX = float(BostonCentroid.x) # -71.08097 longitude
Bos_centerY = float(BostonCentroid.y) # 42.31953 latitude
Bos_centerX32619 = float(BostonCentroid32619.x) # 328520.137058
Bos_centerY32619 = float(BostonCentroid32619.y) # 44.687352e+06 int is 4687351
print("...VARIABLES CREATED")

In [ ]:
# creating shapely polygons of city (no water)
BostonPolygon = BostonOutline.geometry.unary_union
BostonPolygon32619 = BostonOutline32619.geometry.unary_union
print("...POLYGONS CREATED")

In [ ]:
# working with MBTA data
# rename cols
MBTA_Rail_Stop.rename(columns={"STATION":"Station","LINE_BRNCH":"Line"}, inplace=True)
MBTA_Rail_Line.rename(columns={"LINE_BRNCH":"Line"}, inplace=True)
MBTA_RT_Stop.rename(columns={"STATION":"Station","LINE":"Line","ROUTE":"Route"}, inplace=True)
MBTA_RT_Line.rename(columns={"LINE":"Line","ROUTE":"Route"}, inplace=True)
MBTA_Bus_Line.rename(columns={"ROUTE_DESC":"Route","TRIP_HEADSIGN":"Headsign"}, inplace=True)
# change case
MBTA_Rail_Stop['Station'] = MBTA_Rail_Stop['Station'].str.title()
MBTA_Rail_Stop['Line'] = MBTA_Rail_Stop['Line'].str.title()
MBTA_Rail_Line['Line'] = MBTA_Rail_Line['Line'].str.title()
MBTA_RT_Stop['Line'] = MBTA_RT_Stop['Line'].str.title()
MBTA_RT_Line['Line'] = MBTA_RT_Line['Line'].str.title()
# create lon/lat from point geometry
MBTA_Rail_Stop['lon'] = MBTA_Rail_Stop['geometry'].x
MBTA_Rail_Stop['lat'] = MBTA_Rail_Stop['geometry'].y
MBTA_RT_Stop['lon'] = MBTA_RT_Stop['geometry'].x
MBTA_RT_Stop['lat'] = MBTA_RT_Stop['geometry'].y
print("MBTA_Rail_Stop",'Type:', type(MBTA_Rail_Stop), "/ current CRS is:",MBTA_Rail_Stop.crs)
print("MBTA_Rail_Line",'Type:', type(MBTA_Rail_Line), "/ current CRS is:",MBTA_Rail_Line.crs)
print("MBTA_RT_Stop",'Type:', type(MBTA_RT_Stop), "/ current CRS is:",MBTA_RT_Stop.crs)
print("MBTA_RT_Line",'Type:', type(MBTA_RT_Line), "/ current CRS is:",MBTA_RT_Line.crs)
#print("MBTA_Bus_Stop",'Type:', type(MBTA_Bus_Stop), "/ current CRS is:",MBTA_Bus_Stop.crs)
print("MBTA_Bus_Line",'Type:', type(MBTA_Bus_Line), "/ current CRS is:",MBTA_Bus_Line.crs)

### Venue Data Work Begins

Regardless of how large you set the "limit" parameter in your API request, Foursquare's EXPLORE endpoint only returns up to the first 100* recommended venues. Fortunately these results are paginated, so you can use the OFFSET variable to re-run the request and get the next 100, and so on.

<i>*Foursquare's documentation says EXPLORE will return up to 50, but it's returning 100, so...</i>

The SEARCH endpoint will only retrieve 50 results no matter how high you set the limit, and there is no OFFSET parameter. The makeshift solution suggested by the internet is to repeatedly re-run the search with slightly shifted lat/lon coordinates to try to blanket the area, then remove the duplicates. That sounds extra messy, so we'll be sticking with the EXPLORE endpoint.

EXPLORE can use a parameter to limit the type of venues returned, either "categoryId=categorynumber" or "section=sectionname". Category number requests automatically include all subcategories of that category number, so using category number 4d4b7105d754a06374d81259 (Food) will capture all the varieties of restaurant nested under that category. Another alternative is to use one of the section names, which are high level categories like food, drinks, shops, arts, outdoors, sights, and trending. We’ll just use "section=food".

So we'll use EXPLORE with "section=food" and loop thru while incrementing the "offset=" parameter to fill out our universe. The next question is how many loops? One option would be to simply keep incrementing the offset until the request gave an error message, but that seems rather crude. When you put in an API call with an offset so high that there are no more results, rather than responding with a code 400 Bad Request, Foursquare returns a code 200 message stating that nothing was found and indicating the max number of venues meeting the query request. By running the coordinates of each of our 180 census tracts thru the EXPLORE with an excessively high offset, we can pull a list of the total number of venues within our specified radius of each point.

In [ ]:
# adding Neighborhood data to BostonNoWater
City_Geo = pd.merge(BostonNoWater, BPDA_Neighborhoods, how='left', on='NAMELSAD10')       # join on key
City_Geo.drop(columns=['GEOID10_y', 'NAME10_y'], inplace=True)                            # drop duplicate columns
City_Geo.rename(columns={"2014 BPDA Neighborhood":"Neighborhood","GEOID10_x":"GEOID10",   # rename columns
                           "NAME10_x":"NAME10"}, inplace=True)                            # rename columns
City_Geo["Off0"] = 0                                                                      # adding temp offset cols
City_Geo["Off1"] = 100                                                                    # adding temp offset cols
City_Geo["Off2"] = 200                                                                    # adding temp offset cols
City_Geo["Off3"] = 300                                                                    # adding temp offset cols
City_Geo.reset_index(inplace=True, drop=True)
City_Geo.shape

In [ ]:
# creating MaxVenues function
def getMaxVenues(names, latitudes, longitudes):
    max_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url_max = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}&offset=400'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,SECTION,RADIUS,LIMIT)
        maxresults = requests.get(url_max)
        max_list.append([maxresults.json()['response'],name,lat,lng])
    return(max_list)
print("...'getMaxVenues' FUNCTION CREATED")

In [ ]:
# running MaxVenues function
venuesMax = getMaxVenues(names=City_Geo['Neighborhood'],latitudes=City_Geo['INTPTLAT10'],
                         longitudes=City_Geo['INTPTLON10'])
print('...RETRIEVAL COMPLETE')

In [ ]:
# working with MaxVenue
VMax1 = pd.DataFrame(venuesMax)
VMax1.rename(columns={VMax1.columns[0]:"Response",VMax1.columns[1]:"Neighborhood",VMax1.columns[2]:"Input Lat",
                      VMax1.columns[3]:"Input Lon"}, inplace = True)
VMax2=json_normalize(VMax1['Response'])
VMax3=VMax2.merge(VMax1, how='outer', left_index=True, right_index=True)
VMax3.drop(columns=['groups','suggestedFilters.header','suggestedFilters.filters','suggestedBounds.ne.lat',
                   'suggestedBounds.ne.lng','suggestedBounds.sw.lat','suggestedBounds.sw.lng','Response'], inplace=True)
VMax3.rename(columns={"headerLocation":"4SQ Loc", "headerFullLocation":"4SQ Full Loc",
                     "headerLocationGranularity":"Loc Granularity","query":"Query",
                     "totalResults":"Venue Count"},inplace=True)
VMax=VMax3[['Neighborhood','Input Lat','Input Lon','Venue Count','4SQ Loc','4SQ Full Loc','Loc Granularity','Query']]
VMax.sort_values(by=['4SQ Full Loc','Neighborhood'],ignore_index=True, inplace=True)
print("maximum venue count at any input coord is",VMax['Venue Count'].max())
VMax.head()

Okay, the maximum number of venues in the radius of any of our 180 input locations is 250, so it looks like three loops should cover it. We'll use the defined function below and increment the offset at 0, 100, and 200, then combine the results.

In [ ]:
# create "getNearbyVenuesX00" function with Lat+Lng+Offset from input, remainder from basic variables at top
def getNearbyVenuesX00(names, latitudes, longitudes, offset):
    venues_list=[]
    for name, lat, lng, offs in zip(names, latitudes, longitudes, offset):
        # print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}&offset={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,SECTION,RADIUS,LIMIT,offs)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['id'],v['venue']['location']['lat'],
                             v['venue']['location']['lng'],v['venue']['categories'][0]['name'],
                             v['venue']['location']['distance']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Input Neighborhood','Input Latitude','Input Longitude','Venue','Venue_ID',
                             'Venue_Latitude','Venue_Longitude','Venue_Category','Venue_Distance']
    return(nearby_venues)
print("...'getNearbyVenuesX00' FUNCTION CREATED")

In [ ]:
# Foursquare venues, first loop (offset zero)
venues000 = getNearbyVenuesX00(names=City_Geo['Neighborhood'],latitudes=City_Geo['INTPTLAT10'],
                            longitudes=City_Geo['INTPTLON10'],offset=City_Geo['Off0'])
print('...FIRST LOOP RETRIEVAL COMPLETE')
venues000.shape

In [ ]:
# Foursquare venues, second loop (offset 100)
venues100 = getNearbyVenuesX00(names=City_Geo['Neighborhood'],latitudes=City_Geo['INTPTLAT10'],
                            longitudes=City_Geo['INTPTLON10'],offset=City_Geo['Off1'])
print('...SECOND LOOP RETRIEVAL COMPLETE')
venues100.shape

In [ ]:
# Foursquare venues, third loop (offset 200)
venues200 = getNearbyVenuesX00(names=City_Geo['Neighborhood'],latitudes=City_Geo['INTPTLAT10'],
                            longitudes=City_Geo['INTPTLON10'],offset=City_Geo['Off2'])
print('...THIRD LOOP RETRIEVAL COMPLETE')
venues200.shape

Now time to combined the loop results. It's likely that many venues will be within the specified radius of more than one of our 180 census tract coordinates, so after combining we'll sort the results and drop duplicates.

In [ ]:
# assembling Venue results
# removing temporary offset cols
City_Geo.drop(columns=['Off0','Off1','Off2','Off3'], inplace=True)
# merge each of the "venues" offset outputs into one dataframe
venues_int = venues000.append(venues100, ignore_index=True)
venues = venues_int.append(venues200, ignore_index=True)
venues.reset_index(inplace=True, drop=True)
# removing duplicate entries
length_before = len(venues)
venues.sort_values(by=['Venue_ID','Venue_Distance'],     # sorting the data
                       ignore_index=True, inplace=True)  # adding last two arguements resets the index to the new order
# the "subset" parameter specifies which columns to match on, otherwise it matches on all fields
venues.drop_duplicates(subset=['Venue_ID','Venue_Latitude',
                               'Venue_Longitude'], keep='first', ignore_index=True,inplace=True)
length_after = len(venues)
venues['VenLat'] = venues['Venue_Latitude'].astype('object')
venues['VenLon'] = venues['Venue_Longitude'].astype('object')
print("before",length_before,"/ after",length_after)

Twenty-three thousand before, twenty-three hundred after. Yup, there were a few duplicates.

The US Census Bureau's geocoder API will accept a set of coordinates and send a response indicating what census tract those coordinates are in. We'll loop the coordinates of our venues thru the geocoder and assign each to a specific census tract. This particular chunk of code is a bit slow, as it's making twenty-three hundred requests one at a time. The Census Bureau geocoder page will accept bulk upload of CSV files of up to 10000 locations per file, but I decided it was more in the spirit of this assignment to run as much of it within Python as possible, so we'll go for the API. All things being equal, just running the cell and getting a beverage is still probably quicker than downloading the list of coordinates, uploading it to the geocoder webpage, downloading those results, pulling them back into the notebook, and fixing any formatting issues.

In [ ]:
# create "getCensusInfo" function with Lat+Lng from input "venues"
# determines what Census Tract each venue is located in
# benchmark (4) = Public_AR_Current
# vintage (410) = Census2010_Current

def getCensusInfo(ven_ids, lats, lons):
    census_list=[]
    for ven_id, lat, lng in zip(ven_ids, lats, lons):
        # print(ven_id)
        url_batch_geocode = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={}&y={}&benchmark=4&vintage=410&format=json'.format(
            lng,lat)
        census_results = requests.get(url_batch_geocode).json()['result']['geographies']['Census Tracts']
        census_list.append([(ven_id,lat,lng,v['POP100'],v['GEOID'],v['INTPTLAT'],v['NAME'],v['INTPTLON'],
                             v['HU100']) for v in census_results])
    census_info = pd.DataFrame([item for loc_list in census_list for item in loc_list])
    census_info.columns = ['Venue_ID','Venue_Latitude','Venue_Longitude','Population','GEOID','INTPTLAT',
                           'NAME','INTPTLON','Housing_Units']
    return(census_info)
print("...'getCensusInfo' FUNCTION CREATED")

In [ ]:
# running getCensusInfo
census_response = getCensusInfo(ven_ids=venues['Venue_ID'],lats=venues['VenLat'],lons=venues['VenLon'])
print('...CENSUS TRACT RETRIEVAL COMPLETE')
census_response.shape

Now that we have our venues and we have census tract data to go with each, we can eliminate entries that were within our specified radius but are not actually in the City of Boston. Every venue location will now be tagged with a GEOID10 number from the Census Bureau. The format of the number is:

SSCCCTTTTT with **S**tate: 2 digits, **C**ounty: 3 digits, **T**ract: 6 digits.

So a GEOID10 of 25025020101 = State 25, County 025, Tract 020101 (201.01). Massachusetts is state 25, and Suffolk County is county 025, so if it doesn't start "25025" then it's not in Suffolk County MA. Next is filtering out the three communities in Suffolk County that aren't Boston; Chelsea (tracts 1601.01, 1602, 1603, 1604, 1605.01, 1605.02, 1606.01, 1606.02), Revere (tracts 1701, 1702, 1703, 1704, 1705.01, 1705.02, 1706.01, 1707.01, 1707.02, 1708), and Winthrop (tracts 1801.01, 1802, 1803.01, 1804, 1805). We'll pass our geocoded venue data thru a few filters, limiting to just those which start with '25025' to dump the entries from the counties of Middlesex, Essex, and Norfolk, then remove entries starting with '25025160', '25025170', and '25025180' to pull out Chelsea, Revere, and Winthrop.

In [ ]:
# adding Census data to Venue data
venue_census = pd.merge(venues, census_response, how='left', on='Venue_ID')
venue_census.drop(columns=['Venue_Latitude_y','Venue_Longitude_y','Input Latitude',              # drop dupe columns
                           'Input Longitude'], inplace=True)
venue_census.rename(columns={"Venue_Latitude_x":"VenLatNum", "Venue_Longitude_x":"VenLonNum",    # rename columns
                             "GEOID":"GEOID10","NAME":"NAMELSAD10"},inplace=True)
venue_census=venue_census[['Venue_ID','Venue','Venue_Category','VenLat','VenLatNum','VenLon',    # re-ordering
                           'VenLonNum','Population','Housing_Units','GEOID10','NAMELSAD10',
                           'INTPTLAT','INTPTLON']]

# removing non-Boston entries:
vc_length_before = len(venue_census)
venue_census = venue_census[venue_census.GEOID10.str.startswith(('25025'))]                      # drop non-Suffolk
venue_census = venue_census[~venue_census.GEOID10.str.startswith(('25025160'))]                  # drop Chelsea
venue_census = venue_census[~venue_census.GEOID10.str.startswith(('25025170'))]                  # drop Revere
venue_census = venue_census[~venue_census.GEOID10.str.startswith(('25025180'))]                  # drop Winthrop
venue_census.reset_index(inplace=True, drop=True)
vc_length_after = len(venue_census)
print("before",vc_length_before,"/ after",vc_length_after)

This is our final venue count. Next we'll take our cleaned up list of venues and use the associated census tracts to assign the venues to neighborhoods as defined by the Boston Planning & Development Agency.

In [ ]:
# adding Neighborhood data to Venue/Census data
ven_neigh = pd.merge(venue_census, City_Geo, how='inner', on='NAMELSAD10')                          # join on key
ven_neigh.drop(columns=['GEOID10_y','INTPTLAT','INTPTLON'], inplace=True)                           # drop columns
ven_neigh.rename(columns={"Venue_Category":"Venue Category","Venue_ID":"Venue ID",                  # rename columns
                          "GEOID10_x":"GEOID10","Housing_Units":"Housing Units"}, inplace=True)
ven_neigh.reset_index(inplace=True, drop=True)

# adding Broad Category to Venue/Census/Neighborhood data
ven_combined = pd.merge(ven_neigh, Broad_Categories, how='left', on='Venue Category')               # join on key
ven_combined=ven_combined[['Venue ID','Venue','Venue Category','Broad Category','Short Type',       # re-ordering columns
                           'VenLat','VenLatNum','VenLon','VenLonNum','Neighborhood','Population',
                           'Housing Units','GEOID10','NAMELSAD10','FID','OBJECTID','STATEFP10',
                           'COUNTYFP10','TRACTCE10','NAME10','MTFCC10','FUNCSTAT10','ALAND10',
                           'AWATER10','INTPTLAT10','INTPTLON10','Shape_STAr','Shape_STLe',
                           'Shape__Area','Shape__Length','geometry']]
ven_combined.reset_index(inplace=True, drop=True)
ven_combined.shape

### Population Data Work Begins

Using a couple of different parts of the Census Bureau data resources, we'll pull in population, housing, and income info for each of the census tracts in the city. For various purposes, we’ll arrange this into DataFrames on a tract-by-tract basis, a neighborhood-by-neighborhood basis, and a whole-city basis.

In [ ]:
# create "getCensusPop" function with Lat+Lng from input, gets population and housing info for each census tract
def getCensusPop(hoods, lats, lons):
    census_list=[]
    for hood, lat, lng in zip(hoods, lats, lons):
        # print(hood)
        url_batch_geocode = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={}&y={}&benchmark=4&vintage=410&format=json'.format(
            lng,lat)
        census_results = requests.get(url_batch_geocode).json()['result']['geographies']['Census Tracts']
        census_list.append([(hood,lat,lng,v['POP100'],v['GEOID'],v['INTPTLAT'],v['NAME'],v['INTPTLON'],
                             v['HU100']) for v in census_results])
    census_info = pd.DataFrame([item for loc_list in census_list for item in loc_list])
    census_info.columns = ['Neighborhood','Hood_Latitude','Hood_Longitude','Population','GEOID','INTPTLAT',
                           'NAME','INTPTLON','Housing_Units']
    return(census_info)
print("...'getCensusInfo' FUNCTION CREATED")

In [ ]:
# running getCensusPop
census_pop = getCensusPop(hoods=City_Geo['Neighborhood'],lats=City_Geo['INTPTLAT10'],lons=City_Geo['INTPTLON10'])
print('...CENSUS POP RETRIEVAL COMPLETE')
census_pop.shape

In [ ]:
# cleaning up census data
census_pop=census_pop[['Neighborhood','Population','Housing_Units','GEOID','NAME','INTPTLAT',    # re-ordering columns
                       'INTPTLON','Hood_Latitude', 'Hood_Longitude']]
census_pop.rename(columns={"GEOID":"GEOID10", "NAME":"NAMELSAD10",                               # rename columns
                           "Housing_Units":"Housing Units"}, inplace=True)
census_pop.drop(columns=['Hood_Latitude', 'Hood_Longitude'], inplace=True)                       # drop dupe columns
c_length_before = len(census_pop)
census_pop.sort_values(by=['Neighborhood','GEOID10'],         # sorting the data
                           ignore_index=True, inplace=True)   # adding last two args resets the index to the new order
census_pop.drop_duplicates(subset=['GEOID10','NAMELSAD10'],   # removing duplicate entries
                           keep='first', ignore_index=True,   # the "subset" parameter specifies which columns to 
                           inplace=True)                      # match on, otherwise it matches on all fields
c_length_after = len(census_pop)
print("before",c_length_before,"/ after",c_length_after)

In [ ]:
# get income info for each census tract in Suffolk County, variables in /data/2019/acs/acs5/groups/B19013
url_medinc = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,B19013_001E&for=tract:*&in=state:25&in=county:025&key={}'.format(census_key)
results_medinc = requests.get(url_medinc).json()
medinc=pd.DataFrame(results_medinc[1:], columns=results_medinc[0])                          # convert to dataframe
medinc[['NAMELSAD10','County Name','State Name']]=medinc.NAME.str.split(", ",expand=True)   # split col
medinc['Median Household Income'] = medinc['B19013_001E'].astype('float64') 
medinc.rename(columns={"state":"State","county":"County","tract":"TRACTCE10"},inplace=True) # rename columns
medinc=medinc[['NAMELSAD10','TRACTCE10','Median Household Income','State','State Name',     # re-ordering columns
               'County','County Name']]
medinc['Median Household Income'].replace(-666666666,np.nan,inplace=True)                   # replace wonky values
medinc['Median Household Income'].replace(0,np.nan,inplace=True)                            # replace wonky values
medinc.shape

### CensusGeo - Tract
Arranging the Census info based on tract

In [ ]:
# turn "groupby" totals of venues into dataframe
tract_venue_count = ven_combined.groupby('NAMELSAD10', dropna=False)[["Venue ID"]].count().reset_index()
tract_venue_count.rename(columns={"Venue ID":"Venue Count"}, inplace=True) # rename column
tract_venue_count.shape

In [ ]:
# adding Census Population data to City_Geo
tractgeo_a = pd.merge(City_Geo, census_pop, on='NAMELSAD10')                                     # join on key
tractgeo_a.drop(columns=['GEOID10_y','Neighborhood_y'], inplace=True)                            # drop dupe columns
tractgeo_a.rename(columns={"GEOID10_x":"GEOID10","Neighborhood_x":"Neighborhood"}, inplace=True) # rename column
tractgeo_a.reset_index(inplace=True, drop=True)
tStep1 = tractgeo_a.shape
# adding venue count columns
tractgeo_b = pd.merge(tractgeo_a, tract_venue_count, how='left', on='NAMELSAD10')                 # join on key
tractgeo_b.reset_index(inplace=True, drop=True)
tStep2 = tractgeo_b.shape
# adding Census Income data
tractgeo_c = pd.merge(tractgeo_b, medinc, how='left', on='NAMELSAD10')                           # join on key
tractgeo_c.drop(columns=['TRACTCE10_y','INTPTLAT','INTPTLON','State','County'], inplace=True)    # drop dupe columns
tractgeo_c.rename(columns={"TRACTCE10_x":"TRACTCE10"}, inplace=True)                             # rename columns
tractgeo_c.reset_index(inplace=True, drop=True)
tStep3 = tractgeo_c.shape
# creating derived field: population density by tract
# ALAND10 value is square meters, divide by 2,589,988 to convert to square miles 
tractgeo_c['Population Density'] = (tractgeo_c['Population'] / (tractgeo_c['ALAND10'] / 2589988)).round(2)
# creating derived field: venue density by tract
# ALAND10 value is square meters, divide by 2,589,988 to convert to square miles 
tractgeo_c['Venue Density'] = (tractgeo_c['Venue Count'] / (tractgeo_c['ALAND10'] / 2589988)).round(2)
# creating derived field: population per venue by tract
tractgeo_c['Pop per Venue'] = (tractgeo_c['Population'] / tractgeo_c['Venue Count']).round(2)
# creating derived field: population per housing unit by tract
tractgeo_c['Pop per HU'] = (tractgeo_c['Population'] / tractgeo_c['Housing Units']).round(2)
# adding dummy 'pop per ven' value for tracts with 0 venues and large population, pop per ven = pop
tractgeo_c.loc[(tractgeo_c['Pop per Venue'].isnull()) & (tractgeo_c['Population'] > 1000), 'Pop per Venue'] = tractgeo_c['Population']
# addressing excessive values
tractgeo_c['Pop per HU'].replace(np.inf,np.nan,inplace=True)
tractgeo_c['Pop per HU'] = np.where((tractgeo_c['Pop per HU'] > 6), np.nan, tractgeo_c['Pop per HU'])
tStep4 = tractgeo_c.shape
# re-ordering columns
tractgeo=tractgeo_c[['Neighborhood','NAMELSAD10','Population','Population Density','Housing Units',
                     'Pop per HU','Venue Count','Venue Density','Pop per Venue','Median Household Income','FID',
                     'OBJECTID','STATEFP10','State Name','COUNTYFP10','County Name','TRACTCE10','GEOID10','NAME10',
                     'MTFCC10','FUNCSTAT10','ALAND10','AWATER10','INTPTLAT10','INTPTLON10','Shape_STAr',
                     'Shape_STLe','Shape__Area','Shape__Length','geometry']]
tractgeo.reset_index(inplace=True, drop=True)
tractgeo.sort_values(by=['Neighborhood','NAMELSAD10'],ignore_index=True, inplace=True)
tStep5 = tractgeo.shape
print("shape changes:",tStep1,"/",tStep2,"/",tStep3,"/",tStep4,"/",tStep5)

In the calculations above, I found there were a handful of tracts which had significant populations but no venues at all. As we'll see in a subsequent chart, letting these tracts fall to "NaN" (not a number) on the "population per venue" count would be a missed opportunity, so instead for any tract with population greater than 1000 and 0 venues, I assigned a "population per venue" figure equal to the population of the tract.

### CensusGeo - Neighborhood
Arranging the Census info aggregated to neighborhood

In [ ]:
# groupby totals
# turn "groupby" totals of venues into dataframe
hood_venue_count = ven_combined.groupby('Neighborhood', dropna=False)[["Venue ID"]].count().reset_index()
hood_venue_count.rename(columns={"Venue ID":"Venue Count (hood)"}, inplace=True)                  # rename column
# turn "groupby" totals of neighborhoods into dataframe
hood_pop = tractgeo.groupby('Neighborhood', dropna=False)[["Population"]].sum().reset_index()
hood_pop.rename(columns={"Population":"Population (hood)"}, inplace=True)                         # rename column
# turn "groupby" totals of neighborhood land area into dataframe
hood_area = tractgeo.groupby('Neighborhood', dropna=False)[["ALAND10","AWATER10"]].sum().reset_index()
hood_area.rename(columns={"ALAND10":"ALAND10 (hood)","AWATER10":"AWATER10 (hood)"}, inplace=True) # rename column
# turn "groupby" totals of neighborhood land area into dataframe
hood_units = tractgeo.groupby('Neighborhood', dropna=False)[["Housing Units"]].sum().reset_index()
hood_units.rename(columns={"Housing Units":"Housing Units (hood)"}, inplace=True)                 # rename column
print("hood_venue_count",hood_venue_count.shape)
print("hood_pop",hood_pop.shape)
print("hood_area",hood_area.shape)
print("hood_units",hood_units.shape)

In [ ]:
# adding Neighborhood Venue Count data to hoodgeo
hoodgeo_a = pd.merge(BostonHoods, hood_venue_count, how='left', on='Neighborhood')               # join on key
hoodgeo_a.reset_index(inplace=True, drop=True)
hStep1 = hoodgeo_a.shape
# adding Neighborhood Population data to hoodgeo
hoodgeo_b = pd.merge(hoodgeo_a, hood_pop, how='left', on='Neighborhood')                        # join on key
hoodgeo_b.reset_index(inplace=True, drop=True)
hStep2 = hoodgeo_b.shape
# adding Neighborhood Area data to hoodgeo
hoodgeo_c = pd.merge(hoodgeo_b, hood_area, how='left', on='Neighborhood')                       # join on key
hoodgeo_c.drop(columns=['ALAND10 (hood)', 'AWATER10 (hood)'], inplace=True)                     # dropping columns
hoodgeo_c.reset_index(inplace=True, drop=True)
hStep3 = hoodgeo_c.shape
# adding Neighborhood Area data to hoodgeo
hoodgeo_d = pd.merge(hoodgeo_c, hood_units, how='left', on='Neighborhood')                        # join on key
hoodgeo_d.rename(columns={'ALAND10':'ALAND10 (hood)','AWATER10':'AWATER10 (hood)','Shape_STAr':'Shape_STAr (hood)',
                          'Shape_STLe':'Shape_STLe (hood)','Shape__Area':'Shape__Area (hood)',
                          'Shape__Length':'Shape__Length (hood)'}, inplace=True)                  # rename columns
hoodgeo_d.reset_index(inplace=True, drop=True)
hStep4 = hoodgeo_d.shape
# creating derived field: population density by hood
# ALAND10 value is square meters, divide by 2,589,988 to convert to square miles 
hoodgeo_d['Population Density (hood)'] = (hoodgeo_d['Population (hood)'] / 
                                           (hoodgeo_d['ALAND10 (hood)'] / 2589988)).round(2)
# creating derived field: venue density by hood
# ALAND10 value is square meters, divide by 2,589,988 to convert to square miles 
hoodgeo_d['Venue Density (hood)'] = (hoodgeo_d['Venue Count (hood)'] / 
                                      (hoodgeo_d['ALAND10 (hood)'] / 2589988)).round(2)
# creating derived field: population per venue unit by hood
hoodgeo_d['Pop per Venue (hood)'] = (hoodgeo_d['Population (hood)'] / 
                                  hoodgeo_d['Venue Count (hood)']).round(2)
# creating derived field: population per housing unit by hood
hoodgeo_d['Pop per HU (hood)'] = (hoodgeo_d['Population (hood)'] / 
                                  hoodgeo_d['Housing Units (hood)']).round(2)
# addressing excessive values
hoodgeo_d['Population Density (hood)'].replace(np.inf,np.nan,inplace=True)
hoodgeo_d['Venue Density (hood)'].replace(np.inf,np.nan,inplace=True)
hoodgeo_d['Pop per HU (hood)'].replace(np.inf,np.nan,inplace=True)
hoodgeo_d['Pop per HU (hood)'] = np.where((hoodgeo_d['Pop per HU (hood)'] > 6), np.nan, hoodgeo_d['Pop per HU (hood)'])
hStep5 = hoodgeo_d.shape
# re-ordering columns
hoodgeo=hoodgeo_d[['Neighborhood','Population (hood)','Population Density (hood)','Housing Units (hood)',
                   'Pop per HU (hood)','Venue Count (hood)','Venue Density (hood)','Pop per Venue (hood)',
                   'ALAND10 (hood)','AWATER10 (hood)','Shape_STAr (hood)','Shape_STLe (hood)','Shape__Area (hood)',
                   'Shape__Length (hood)','geometry']]
hoodgeo.reset_index(inplace=True, drop=True)
hoodgeo.sort_values(by=['Neighborhood'],ignore_index=True, inplace=True)
hStep6 = hoodgeo.shape
print("shape changes:",hStep1,"/",hStep2,"/",hStep3,"/",hStep4,"/",hStep5,"/",hStep6)

### CensusGeo - City
Arranging the Census info aggregated to the entire city

In [ ]:
# working with city-level data
citygeo_a = BostonOutline
citygeo_a['Population (city)'] = tractgeo['Population'].sum()
citygeo_a['Venue Count (city)'] = ven_combined['Venue ID'].count()
citygeo_a['Housing Units (city)'] = tractgeo['Housing Units'].sum()
citygeo_a.rename(columns={"ALAND10":"ALAND10 (city)","AWATER10":"AWATER10 (city)"}, inplace=True) # rename column
# creating derived field: population density for city
# ALAND10 value is square meters, divide by 2,589,988 to convert to square miles 
citygeo_a['Population Density (city)'] = (citygeo_a['Population (city)'] / 
                                          (citygeo_a['ALAND10 (city)'] / 2589988)).round(2)
# creating derived field: venue density for city
# ALAND10 value is square meters, divide by 2,589,988 to convert to square miles 
citygeo_a['Venue Density (city)'] = (citygeo_a['Venue Count (city)'] / 
                                     (citygeo_a['ALAND10 (city)'] / 2589988)).round(2)
# creating derived field: population per venue for city
citygeo_a['Pop per Venue (city)'] = (citygeo_a['Population (city)'] / 
                                     citygeo_a['Venue Count (city)']).round(2)
# creating derived field: population per housing unit for city
citygeo_a['Pop per HU (city)'] = (citygeo_a['Population (city)'] / 
                                  citygeo_a['Housing Units (city)']).round(2)
citygeo=citygeo_a[['Population (city)','Population Density (city)','Housing Units (city)','Pop per HU (city)',
                   'Venue Count (city)','Venue Density (city)','Pop per Venue (city)','ALAND10 (city)',
                   'AWATER10 (city)','Shape_STAr','Shape_STLe','Shape__Area','Shape__Length','geometry']]
citygeo.shape

### Merge to Venue Data

In [ ]:
# adding all Census data back into venue list
# adding tract data in
venue_master_a = pd.merge(ven_combined, tractgeo, how='left', on='NAMELSAD10')                       # join on key
vStep1 = venue_master_a.shape
venue_master_a.rename(columns={'Neighborhood_x':'Neighborhood','Population_x':'Population',          # rename columns
                              'Housing Units_x':'Housing Units','GEOID10_x':'GEOID10',
                              'FID_x':'FID','OBJECTID_x':'OBJECTID','STATEFP10_x':'STATEFP10',
                              'COUNTYFP10_x':'COUNTYFP10','TRACTCE10_x':'TRACTCE10','NAME10_x':'NAME10',
                              'MTFCC10_x':'MTFCC10','FUNCSTAT10_x':'FUNCSTAT10','ALAND10_x':'ALAND10',
                              'AWATER10_x':'AWATER10','INTPTLAT10_x':'INTPTLAT10',
                              'INTPTLON10_x':'INTPTLON10'}, inplace=True)
venue_master_a.drop(columns=['Neighborhood_y','Population_y','Housing Units_y','FID_y','OBJECTID_y', # drop dupe columns
                             'STATEFP10_y','COUNTYFP10_y','TRACTCE10_y','GEOID10_y','NAME10_y',
                             'MTFCC10_y','FUNCSTAT10_y','ALAND10_y','AWATER10_y','INTPTLAT10_y',
                             'INTPTLON10_y','Shape_STAr_y','Shape_STLe_y','Shape__Area_y',
                             'Shape__Length_y','Shape_STAr_x','Shape_STLe_x','Shape__Area_x',
                             'Shape__Length_x','geometry_y','geometry_x'], inplace=True)
venue_master_a.reset_index(inplace=True, drop=True)
vStep2 = venue_master_a.shape
# adding neighborhood data in
venue_master_b = pd.merge(venue_master_a, hoodgeo, how='left', on='Neighborhood')                    # join on key
venue_master_b.drop(columns=['geometry'], inplace=True)                                              # drop column
venue_master_b.reset_index(inplace=True, drop=True)
vStep3 = venue_master_b.shape
venue_master_c = venue_master_b
# adding citywide total cols
venue_master_c['Population (city)'] = citygeo['Population (city)'].sum()
venue_master_c['Venue Count (city)'] = citygeo['Venue Count (city)'].sum()
venue_master_c['Housing Units (city)'] = citygeo['Housing Units (city)'].sum()
venue_master_c['ALAND10 (city)'] = citygeo['ALAND10 (city)'].sum()
venue_master_c['AWATER10 (city)'] = citygeo['AWATER10 (city)'].sum()
venue_master_c['Population Density (city)'] = citygeo['Population Density (city)'].sum()
venue_master_c['Venue Density (city)'] = citygeo['Venue Density (city)'].sum()
venue_master_c['Pop per Venue (city)'] = citygeo['Pop per Venue (city)'].sum()
venue_master_c['Pop per HU (city)'] = citygeo['Pop per HU (city)'].sum()
venue_master_c.reset_index(inplace=True, drop=True)
vStep4 = venue_master_c.shape
venue_master=venue_master_b[['Venue ID','Venue','Venue Category','Broad Category','Short Type',      # ordering columns
                             'VenLat','VenLatNum','VenLon','VenLonNum','Population',
                             'Population Density','Housing Units','Pop per HU','Venue Count',
                             'Venue Density','Pop per Venue','Median Household Income',
                             'GEOID10','NAMELSAD10','FID','OBJECTID','STATEFP10','State Name',
                             'COUNTYFP10','County Name','TRACTCE10','NAME10','MTFCC10','FUNCSTAT10',
                             'ALAND10','AWATER10','INTPTLAT10','INTPTLON10','Neighborhood',
                             'Population (hood)','Population Density (hood)','Housing Units (hood)',
                             'Pop per HU (hood)','Venue Count (hood)','Venue Density (hood)',
                             'Pop per Venue (hood)','ALAND10 (hood)','AWATER10 (hood)','Population (city)',
                             'Population Density (city)','Housing Units (city)','Pop per HU (city)',
                             'Venue Count (city)','Venue Density (city)','Pop per Venue (city)',
                             'ALAND10 (city)','AWATER10 (city)']]
venue_master.reset_index(inplace=True, drop=True)
vStep5 = venue_master.shape
print("shape changes:",vStep1,"/",vStep2,"/",vStep3,"/",vStep4,"/",vStep5)

### <span style='color:white;background:red;font-family:Helvetica'>&nbsp;Looking At The Data&nbsp;</span>
Now we start looking at these piles of data we've accumulated and arranged, getting a sense of the shape of the city and it's neighborhoods. We'll start with a list of the neighborhoods with population, housing units, and total number of venues in each.

In [ ]:
# groupby of high level neighborhood details
tractgeo.groupby('Neighborhood', dropna=False)[["Population", "Housing Units",
                                                "Venue Count"]].sum().style.format('{0:,.0f}')

No, I don't know where those 535 people on the Harbor Islands are living, given the absence of housing units. The Census Bureau defines a housing unit as "*A house, an apartment, a mobile home or trailer, a group of rooms, or a single room occupied as separate living quarters, or if vacant, intended for occupancy as separate living quarters. Separate living quarters are those in which the occupants live separately from any other individuals in the building and which have direct access from outside the building or through a common hall. For vacant units, the criteria of separateness and direct access are applied to the intended occupants whenever possible.*" So perhaps they sleep on boats...

Next we'll crunch some numbers on the categories of venues we pulled from Foursquare.

In [ ]:
venue_master.groupby('Venue Category', dropna=False)["Venue ID"].count().sort_values(ascending=False)

As we see above, Foursquare lists 90 categories for food venues in Boston. To try to flatten that down to something more manageable, I've introduced a higher level categorization called "Broad Category". The broad category groupings I've made are listed in the table below. One could quibble about what should go into what category, but this is my working set. Part of my motivation is that Foursquare allows varying degrees of specificity in categorization, resulting in 34 venues just being categorized as "Asian", 80 more specifically labeled as "Chinese", and then some labeled even more specifically as "Szechuan". Likewise there are "Japanese", then also "Sushi", "Udon", etc. The uneven degrees of specificity / granularity prove bothersome when trying to get accurate aggregate views; broadly speaking, East Asian restaurants are one the most common types in Boston, but split out into a dozen subgroups buries them in any ranking list.

Another issue is that a rather unhelpful number of entries in Foursquare's data have generic categorizations like "Food" or "Restaurant". There are also a decent number of "Café" entries, which traditionally might mean "Coffeehouse", but upon closer inspection the label has been applied to everything from coffee shops to greasy spoon diners. I've lumped all these together in as "General".

In [ ]:
# turn "groupby" totals of broad categories into dataframe
ven_broad_cat = venue_master.groupby('Broad Category', dropna=False)[["Venue ID"]].count()
ven_broad_cat = ven_broad_cat.sort_values(by=['Venue ID'], ascending=False).reset_index()
ven_broad_cat.rename(columns={"Venue ID":"Venues"}, inplace=True) # rename column

# making Venue Cat to Broad Cat table
cat_cat_a = venue_master[['Broad Category','Venue Category']]
cat_cat_a.drop_duplicates(subset=['Broad Category','Venue Category'], keep='first', ignore_index=True,inplace=True)
cat_cat_b = cat_cat_a.pivot_table(index=['Broad Category'],values='Venue Category',aggfunc=lambda x: ', '.join(x))
cat_cat_b.reset_index(inplace=True, drop=False)
cat_cat_c = pd.merge(cat_cat_b, ven_broad_cat, how='left', on='Broad Category')
cat_cat_c = cat_cat_c[['Broad Category','Venues','Venue Category']]
cat_styler = cat_cat_c.style.set_properties(subset=['Venue Category'],**{'text-align': 'left'})\
.set_properties(subset=['Broad Category'],**{'text-align': 'left', 'width': '130px'})
cat_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
display(cat_styler.hide_index())

If you prefer a more colorful look at the distribution of broad categories, here's a waffle chart instead...

In [ ]:
# waffle chart of categories
# code below will pass a colormap to waffle that pywaffle won't take natively
# n_categories=ven_broad_cat.shape[0]
# colors=[plt.cm.gist_ncar(i/float(n_categories)) for i in range(n_categories)]
fig=plt.figure(FigureClass=Waffle,
               values=ven_broad_cat['Venues'], # / 5, # adding number makes each square worth x number of entries
               rows=25,                               # can specify rows, cols, both, or neither
               #columns=70,                           # can specify rows, cols, both, or neither
               block_arranging_style='normal',      # options are 'normal', 'snake', 'new-line'
               vertical=False,                        # orientation of graph
               cmap_name="tab20",                     # use either "cmap_name" or "colors"
               #colors=colors,                        # use either "cmap_name" or "colors"
               facecolor='#EEEEEE',                   # background color
               #icon_size=20,icon_legend=True,        # when using Icons rather than squares
               labels=[l for l in ven_broad_cat['Broad Category']],
               #labels=[fill(l, 10) for l in ven_broad_cat['Broad Category']],
               #labels=["{0} ({1})".format(n[0], n[1]) for n in ven_broad_cat[['Broad Category', 'Venues']].itertuples()],
               legend={'loc': 'upper left', 'bbox_to_anchor': (0,0), 'ncol': 11, 'framealpha': 0},
               #legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
               title={'label':'Broad Categories of Boston Venues','loc':'left','fontdict':{'fontsize': 20}},
               tight=True,
               figsize=(20,16))                       # width, height, supposedly in inches
plt.show()

In [ ]:
# create version DF verison of hoodgeo GDF for charts
hoodgeo_temp = pd.DataFrame(hoodgeo)
hoodgeo_temp.sort_values(by=['Pop per Venue (hood)','Neighborhood'],ignore_index=True, inplace=True)
hoodgeo_temp.reset_index(inplace=True, drop=True)
TX = float(citygeo['Pop per Venue (city)'])
for i in range(35):
    hoodgeo_temp['padding'] = ['3' if x < TX else '-39' for x in hoodgeo_temp['Pop per Venue (hood)']]
hoodgeo_temp['padding'] = hoodgeo_temp.padding.astype(float)
hoodgeo_temp.shape

In [ ]:
def add_value_labels(ax, spacing=5):     # Add labels to the end of each bar in a bar chart
# Arguments:
#   ax (matplotlib.axes.Axes): The matplotlib object containing the axes of the plot to annotate
#   spacing (int): The distance between the labels and the bars
    for rect in rects:                   # Get X and Y placement of label from rect
        x_value = rect.get_width()
        y_value = rect.get_y() + rect.get_height() / 2
        space = 5                        # Number of points between bar and label
        ha = 'left'                      # Vertical alignment for positive values
#        if x_value < 0:                  # If value of bar is negative: Place label left of bar
        if x_value < citygeo['Pop per Venue (city)'].loc[0]:
            space *= -1                  # Invert space to place label to the left
            ha = 'right'                 # Horizontally align label at right
        label = "{:.2f}".format(x_value) # Use X value as label and format number with one decimal place
        plt.annotate(                    # Create annotation
            label,                       # Use `label` as label
            (x_value, y_value),          # Place label at end of the bar
            xytext=(space, 0),           # Horizontally shift label by `space`
            textcoords="offset points",  # Interpret `xytext` as offset in points
            va='center',                 # Vertically center label
            ha=ha)                       # Horizontally align label differently for positive and negative values
print("...FUNCTION CREATED")

In [ ]:
# Neighborhood by Population per Venue conventional bar chart
fig, ax = plt.subplots(figsize=(14, 10))
hbars = ax.barh(hoodgeo_temp['Neighborhood'], hoodgeo_temp['Pop per Venue (hood)'], height=0.8, align='center')
ax.set_yticks(hoodgeo_temp['Neighborhood'])
ax.set_yticklabels(hoodgeo_temp['Neighborhood'])
ax.set_xlabel('Population')
ax.set_title('Neighborhoods by Population per Venue')
ax.bar_label(hbars, padding=3, fmt='%.2f')
# ax.invert_yaxis()  # labels read top-to-bottom
plt.show()

This is the where the metaphorical lightbulb went on for me on how to approach this "where to put a restaurant" problem. As you'll see in subsequent maps, the Census Bureau data supplied land area and population for each tract, from which we can calculate population density. Add in the Foursquare data and we can also calculate venue density. Looking at areas of the city with higher population density and lower venue density got me thinking of how to tie those two stats together, and what I came up with was Population per Venue, the population of a tract or neighborhood divided by the number venues in a tract or neighborhood. Where in the city are there way more people than venues? I took the chart above and re-worked it to show the bars as being above and below the city-wide mean value:

In [ ]:
# Neighborhood by Population per Venue diverging bar chart
fig, axp = plt.subplots(figsize=(14, 10))
plt.hlines(y=hoodgeo_temp['Neighborhood'],
           xmin=citygeo['Pop per Venue (city)'],                           # auto-set to citywide average
           xmax=hoodgeo_temp['Pop per Venue (hood)'],
           colors='red', alpha=1, linewidth=18)                            # Plotting the horizontal lines
rects = ax.patches
plt.axvline(citygeo['Pop per Venue (city)'].loc[0], color='red', linewidth=0.8)
add_value_labels('Pop per Venue (hood)',1)
plt.gca().set(ylabel=None, xlabel='Population')                            # Setting the labels of x-axis and y-axis
plt.title('Neighborhoods by Population per Venue', fontdict={'size': 20})  # Title of Bar Chart
plt.grid(linestyle='--', alpha=0.5)                                        # Optional grid layout
plt.show()                                                                 # Displaying the Diverging Bar Chart

The red dividing line in the middle represents the city-wide average for people per venue, 370.04. Those in the bottom half have less people per venue, those in the top half have more people per venue. One thing which stands out immediately is that the neighborhoods in the southwest of the city (Mattapan, Hyde Park, Roxbury, Dorchester, West Roxbury) have by far the highest Population per Venue numbers, or to put it another way, far fewer venues per capita, meaning this is the area of the city is under-served in terms of restaurants and should be where we focus for placing a new venue. But first, some maps...

### Maps

In [ ]:
# converting main DataFrames to GeoJSON
tractgeo['Population'].replace(0,np.nan,inplace=True)
tractgeoJSON = tractgeo.to_json()                     # convert dataframe to json
print("...JSON (tractgeo) created")
hoodgeo['Population (hood)'].replace(0,np.nan,inplace=True)
hoodgeoJSON = hoodgeo.to_json()                       # convert dataframe to json
print("...JSON (hoodgeo) created")

It might be useful to select a location that isn't already surrounded by existing venues, so we'll use GeoPandas to add a  buffer around every venue in Boston. Adding circles to a Folium map is easy, however there are a couple of drawbacks to going down that path. First, it just looks messy; the cumulative opacity of overlapping circles in areas with high venue density creates areas that are completely dark and blotted out. Second, it results only in viewable decoration on a map, the buffer zone that results isn't a separate geo-object that you can perform operations with (such as programatically checking if coordinates fall within the shadow of the buffer zone, etc). Thus rather than just using Folium to project this buffer zone round venues, we'll create an object representing the buffer zone. Before that, an aside about Coordinate Reference Systems (CRS)...

Although Python and Folium do a decent job of not burying you in detail when it's not needed, some geodata transformations do require knowing the EPSG number of the CRS your geodata is projected in. EPSG stands for European Petroleum Survey Group, which no longer exists, but their survey data has evolved into a public repository of spatial reference systems, geodetic datums, etc. EPSG numbers have become the de facto industry standard for labeling coordinate reference systems, used by most GIS systems and libraries, including Folium and GeoPandas.

A common standard CRS is EPSG:4326, which uses WGS1984 for both datum and ellipsoid. WGS means it's the World Geodetic System, a coordinate standard used in cartography and navigation which is maintained by the US National Geospatial-Intelligence Agency (NGA). The latest rev is WGS84, first published in 1984 and last revised in 2014. WGS84 is the reference coordinate system used by the Global Positioning System (GPS) and is also the base used by Folium as a default. Unfortunately WGS84 has one drawback when it comes to our buffer zone plans, which I'll get to in a moment.

The GeoPandas geometry.buffer method to create buffers around locations uses a single float64 as input, with no means to specify what unit of measure that number refers to. Instead it uses whatever unit of measure is specified by the CRS which the geodata is projected in. Where this becomes problematic is that EPSG:4326 uses degrees as a unit of measure. Degrees of latitude are fairly consistent, 1° equals approximately 69.1 miles. Longitude is another story, 1° of longitude ranges from 69.1 miles at the equator to 0 miles at the poles. At Boston's coordinates, 1° of latitude and longitude is approximately 69 miles wide by 51 miles tall. Given that geometry.buffer only takes one number as the distance parameter, any buffer made in Boston in the WGS84 CRS will be oval shaped. And given the scale of 1°, getting a reasonable buffer of 1000 feet would also be... some really tiny number I'm not going to bother figuring out.

GeoPandas supports re-projecting from one CRS to another, so for the purposes of adding the buffer to each point we'll switch to a different CRS. EPSG:2249 uses North American Datum 1983 (NAD83) and ellipsoid GRS 1980. NAD83 is from the US Defense Mapping Agency and uses one survey foot as the standard of measure, which makes getting a 1000 foot buffer from geometry.buffer simply a matter of putting in '1000' as the distance. It isn't necessary to re-project it back from EPSG:2249 to EPSG:4326, as Folium will do that for us when reading the data.

The first step is taking a copy of the 'venue_master' dataframe and stripping it to just a few fields, creating a geodataframe from it in CRS EPSG:4326, and converting it to EPSG:2249. This gives us a plot of all the venue locations in a CRS that uses feet as a unit of measure:

In [ ]:
# creating plot of Venues for CRS transform
ven_coords = venue_master[['Venue ID','VenLat','VenLon']]
# converts the list of coords in 'ven_coords' into point geometry, object from DataFrame to GeoDataFrame
# Note that geopandas does lat/lon backwards from conventional wisdom, as lon(x) and lat(y)
ven_coords_gdf = gpd.GeoDataFrame(ven_coords, 
                                        geometry=gpd.points_from_xy(ven_coords.VenLon, ven_coords.VenLat),
                                        crs=4326)
# converting the 'gdf' list of point geometry to a CRS that uses feet as UoM
ven_coords_gdf_projected2249 = ven_coords_gdf.to_crs("epsg:2249")
print('Type:', type(ven_coords_gdf_projected2249), "/ current CRS is:",ven_coords_gdf_projected2249.crs)
ven_coords_gdf_projected2249.plot()

The next step is creating the buffer zone around existing venues using geometry.buffer, then using geometry.unary_union to merge these ~1700 circles into a single multipolygon object. We'll make it projected into a few different CRS in case we need them down the line.

In [ ]:
# creating buffer polygons around the items in the list of point geometry, converts from GeoDataFrame to GeoSeries
ven_locs_buffer2249 = ven_coords_gdf_projected2249.geometry.buffer(1000) # UoM for 2249 is feet, 820.21ft = 250m
ven_locs_buffer32619 = ven_locs_buffer2249.to_crs(epsg=32619)            # UoM for 32619 is meters
ven_locs_buffer = ven_locs_buffer2249.to_crs(epsg=4326)                  # UoM for 4326 is degrees
print('ven_locs_buffer2249 Interim Type:', type(ven_locs_buffer2249), "/ current CRS is:",ven_locs_buffer2249.crs)
print('ven_locs_buffer32619 Interim Type:', type(ven_locs_buffer32619), "/ current CRS is:",ven_locs_buffer32619.crs)
print('ven_locs_buffer (4326) Interim Type:', type(ven_locs_buffer), "/ current CRS is:",ven_locs_buffer.crs)
# merging the individual circles into a single shapley MultiPolygon
ven_locs_buffer_union2249 = ven_locs_buffer2249.geometry.unary_union
ven_locs_buffer_union32619 = ven_locs_buffer32619.geometry.unary_union
ven_locs_buffer_union = ven_locs_buffer.geometry.unary_union
print('ven_locs_buffer_union2249 New Type:', type(ven_locs_buffer_union2249))
ven_locs_buffer_union2249

Then we take the new multipolygon we've created, and convert it back to dataframe then geodataframe, and we're ready to use it in a map layer:

In [ ]:
# convert the Shapely MultiPolygon result to DataFrame
vlb_union_DF = pd.DataFrame([ven_locs_buffer_union2249]).transpose()
vlb_union_DF.rename(columns={0:"geometry",}, inplace=True)
# make GeoDataFrame from DataFrame, assign CRS
vlb_union_GDF = gpd.GeoDataFrame(vlb_union_DF, crs=2249)
print('Type:', type(vlb_union_GDF), "/ current CRS is:",vlb_union_GDF.crs)

Next we'll take these polygons and all the various bits of venue data, census data, buffer data, etc and create the various map layers and feature groups that will be applied to subsequent maps:

In [ ]:
# creating map layers
# create venue marker map layer
fg_VenMarkers = folium.FeatureGroup('Venues',show=False,z_index_offset=1000)
for lat, lng, venue, category in zip(venue_master['VenLat'],venue_master['VenLon'],venue_master['Venue'],
                                     venue_master['Venue Category']):
    label = '{} ({})'.format(venue,category)
    label = folium.Popup(label, parse_html=True, max_width=120)
    folium.CircleMarker([lat, lng],radius=3,popup=label,color='black',fill=True,fill_color='gray',fill_opacity=0.5,
                        parse_html=False,name="Venues").add_to(fg_VenMarkers)
# create venue buffer zone map layer
VenBuffer = folium.Choropleth(geo_data=vlb_union_GDF,name="Buffer Zone",
                              fill_color="Gray",fill_opacity=0.5,line_opacity=0.2,nan_fill_opacity=0,
                              control=True,show=False,legend_name="Buffer Zone")
# create neighborhood map layer
MapHoods = folium.Choropleth(geo_data=hoodgeoJSON, name="Neighborhoods",fill_color="lightblue",fill_opacity=0.5,
                             line_color="darkblue",line_opacity=0.9,line_weight=1,nan_fill_opacity=0,
                             control=True,show=False,highlight=True,legend_name="Neighborhood")
folium.features.GeoJsonPopup(fields=['Neighborhood'],labels=False).add_to(MapHoods.geojson)
# create tract map layer
MapTract = folium.Choropleth(geo_data=tractgeoJSON, name="Census Tracts",fill_color="lightblue",fill_opacity=0.5,
                             line_color="blue",line_opacity=0.9,line_weight=1,nan_fill_opacity=0,
                             control=True,show=False,highlight=True,legend_name="Census Tracts")
folium.features.GeoJsonPopup(fields=['Neighborhood','Population','Housing Units','NAMELSAD10',
                                     'Pop per HU']).add_to(MapTract.geojson)
# create population density by tract map layer
MapPopDen = folium.Choropleth(geo_data=tractgeoJSON,name="Population Density (mi²)",data=tractgeo,bins=8,
                              columns=["NAMELSAD10", "Population Density"],key_on="feature.properties.NAMELSAD10",
                              fill_color="YlOrRd",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,
                              show=False,highlight=True,legend_name="Population per Square Mile")
folium.features.GeoJsonPopup(fields=['Population Density','NAMELSAD10']).add_to(MapPopDen.geojson)
folium.features.GeoJson(tractgeoJSON,name='Labels', # adds hover labels
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['Population Density'],
                                                               aliases = ['Population per Square Mile'],labels=True,
                                                               sticky=False)).add_to(MapPopDen.geojson)
for key in MapPopDen._children:
    if key.startswith('color_map'):
        del(MapPopDen._children[key])
# create population density by neighborhood map layer
MapPopDenH = folium.Choropleth(geo_data=hoodgeoJSON,name="Population Density (mi²)",data=hoodgeo,bins=8,
                               columns=["Neighborhood", "Population Density (hood)"],
                               key_on="feature.properties.Neighborhood",fill_color="YlOrRd",
                               fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,highlight=True,
                               legend_name="Population per Square Mile")
folium.features.GeoJsonPopup(fields=['Population Density (hood)','Neighborhood']).add_to(MapPopDenH.geojson)
folium.features.GeoJson(hoodgeoJSON,name='Labels', # adds hover labels
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['Population Density (hood)'],
                                                               aliases = ['Population per Square Mile'],labels=True,
                                                               sticky=False)).add_to(MapPopDenH.geojson)
for key in MapPopDenH._children:
    if key.startswith('color_map'):
        del(MapPopDenH._children[key])
# create population by tract map layer
MapPop = folium.Choropleth(geo_data=tractgeoJSON,name="Population",data=tractgeo,bins=8,
                           columns=["NAMELSAD10", "Population"],key_on="feature.properties.NAMELSAD10",
                           fill_color="PuRd",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                           highlight=True,legend_name="Population")
folium.features.GeoJsonPopup(fields=['Population','NAMELSAD10']).add_to(MapPop.geojson)
folium.features.GeoJson(tractgeoJSON,name='Labels', # adds hover labels
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['Population'],aliases = ['Population'],
                                                                labels=True,sticky=False)).add_to(MapPop.geojson)
for key in MapPop._children:
    if key.startswith('color_map'):
        del(MapPop._children[key])
# create population by neighborhood map layer
MapPopH = folium.Choropleth(geo_data=hoodgeoJSON,name="Population",data=hoodgeo,bins=8,
                            columns=["Neighborhood", "Population (hood)"],key_on="feature.properties.Neighborhood",
                            fill_color="PuRd",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                            highlight=True,legend_name="Population (hood)")
folium.features.GeoJsonPopup(fields=['Population (hood)','Neighborhood']).add_to(MapPopH.geojson)
folium.features.GeoJson(hoodgeoJSON,name='Labels', # adds hover labels
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['Population (hood)'],
                                                               aliases = ['Population'],
                                                               labels=True,sticky=False)).add_to(MapPopH.geojson)
for key in MapPopH._children:
    if key.startswith('color_map'):
        del(MapPopH._children[key])
# create venues by tract map layer
MapVC = folium.Choropleth(geo_data=tractgeoJSON,name="Venue Count",data=tractgeo,bins=8,
                          columns=["NAMELSAD10", "Venue Count"],key_on="feature.properties.NAMELSAD10",
                          fill_color="PuBuGn",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                          highlight=True,legend_name="Venue Count")
folium.features.GeoJsonPopup(fields=['Venue Count','NAMELSAD10']).add_to(MapVC.geojson)
#folium.features.GeoJson(tractgeoJSON,name='Labels', # adds hover labels
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['Venue Count'],aliases = ['Venue Count'],
#                                                                labels=True,sticky=False)).add_to(MapVC.geojson)
for key in MapVC._children:
    if key.startswith('color_map'):
        del(MapVC._children[key])
# create venues by neighborhood map layer
MapVCH = folium.Choropleth(geo_data=hoodgeoJSON,name="Venue Count",data=hoodgeo,bins=8,
                           columns=["Neighborhood", "Venue Count (hood)"],key_on="feature.properties.Neighborhood",
                           fill_color="PuBuGn",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                           highlight=True,legend_name="Venue Count (hood)")
folium.features.GeoJsonPopup(fields=['Venue Count (hood)','Neighborhood']).add_to(MapVCH.geojson)
#folium.features.GeoJson(hoodgeoJSON,name='Labels', # adds hover labels
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['Venue Count (hood)'],
#                                                               aliases = ['Venue Count'],
#                                                               labels=True,sticky=False)).add_to(MapVCH.geojson)
for key in MapVCH._children:
    if key.startswith('color_map'):
        del(MapVCH._children[key])
# create venue density by tract map layer
MapVD = folium.Choropleth(geo_data=tractgeoJSON,name="Venue Density (mi²)",data=tractgeo,bins=8,
                          columns=["NAMELSAD10", "Venue Density"],key_on="feature.properties.NAMELSAD10",
                          fill_color="RdPu",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                          highlight=True,legend_name="Venue Density")
folium.features.GeoJsonPopup(fields=['Venue Density','NAMELSAD10']).add_to(MapVD.geojson)
#folium.features.GeoJson(tractgeoJSON,name='Labels', # adds hover labels
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['Venue Density'],aliases = ['Venue Density'],
#                                                                labels=True,sticky=False)).add_to(MapVD.geojson)
for key in MapVD._children:
    if key.startswith('color_map'):
        del(MapVD._children[key])
# create venue density by neighborhood map layer
MapVDH = folium.Choropleth(geo_data=hoodgeoJSON,name="Venue Density (mi²)",data=hoodgeo,bins=8,
                           columns=["Neighborhood", "Venue Density (hood)"],key_on="feature.properties.Neighborhood",
                           fill_color="RdPu",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                           highlight=True,legend_name="Venues per Square Mile")
folium.features.GeoJsonPopup(fields=['Venue Density (hood)','Neighborhood']).add_to(MapVDH.geojson)
#folium.features.GeoJson(hoodgeoJSON,name='Labels', # adds hover labels
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['Venue Density (hood)'],
#                                                               aliases = ['Venues per Square Mile'],
#                                                               labels=True,sticky=False)).add_to(MapVDH.geojson)
for key in MapVDH._children:
    if key.startswith('color_map'):
        del(MapVDH._children[key])
# create population per venue by tract map layer
MapPV = folium.Choropleth(geo_data=tractgeoJSON,name="Population per Venue",data=tractgeo,bins=8,
                          columns=["NAMELSAD10", "Pop per Venue"],key_on="feature.properties.NAMELSAD10",
                          fill_color="OrRd",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                          highlight=True,legend_name="Population per Venue")
folium.features.GeoJsonPopup(fields=['Pop per Venue','NAMELSAD10']).add_to(MapPV.geojson)
#folium.features.GeoJson(tractgeoJSON,name='Labels', # adds hover labels
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['Venue Density'],aliases = ['Venue Density'],
#                                                                labels=True,sticky=False)).add_to(MapPV.geojson)
for key in MapPV._children:
    if key.startswith('color_map'):
        del(MapPV._children[key])
# create population per venue by neighborhood map layer
MapPVH = folium.Choropleth(geo_data=hoodgeoJSON,name="Population per Venue",data=hoodgeo,bins=8,
                          columns=["Neighborhood", "Pop per Venue (hood)"],key_on="feature.properties.Neighborhood",
                          fill_color="OrRd",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                          highlight=True,legend_name="Population per Venue")
folium.features.GeoJsonPopup(fields=['Pop per Venue (hood)','Neighborhood']).add_to(MapPVH.geojson)
#folium.features.GeoJson(hoodgeoJSON,name='Labels', # adds hover labels
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['Venue Density (hood)'],
#                                                               aliases = ['Venue Density'],
#                                                               labels=True,sticky=False)).add_to(MapPVH.geojson)
for key in MapPVH._children:
    if key.startswith('color_map'):
        del(MapPVH._children[key])
# create median income by tract map layer
MapPopInc = folium.Choropleth(geo_data=tractgeoJSON,name="Median Household Inc",data=tractgeo,bins=8,
                              columns=["NAMELSAD10","Median Household Income"],key_on="feature.properties.NAMELSAD10",
                              fill_color="Greens",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                              highlight=True,legend_name="Median Household Income")
folium.features.GeoJsonPopup(fields=['Median Household Income','NAMELSAD10']).add_to(MapPopInc.geojson)
#folium.features.GeoJson(tractgeoJSON,name='Labels', # adds hover labels
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['Median Household Income'],
#                                                                aliases = ['Median Household Income'],
#                                                                labels=True,sticky=False)).add_to(MapPopInc.geojson)
for key in MapPopInc._children:
    if key.startswith('color_map'):
        del(MapPopInc._children[key])
# create population per housing unit by tract map layer
MapPopPerHU = folium.Choropleth(geo_data=tractgeoJSON,name="Population per Housing Unit",data=tractgeo,bins=8,
                                columns=["NAMELSAD10","Pop per HU"],key_on="feature.properties.NAMELSAD10",
                                fill_color="PuBu",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                                highlight=True,legend_name="Population per Housing Unit")
folium.features.GeoJsonPopup(fields=['Pop per HU','NAMELSAD10']).add_to(MapPopPerHU.geojson)
folium.features.GeoJson(tractgeoJSON,name='Labels', # adds hover labels
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['Pop per HU'],
                                                                aliases = ['Population per Housing Unit'],
                                                                labels=True,sticky=False)).add_to(MapPopPerHU.geojson)
for key in MapPopPerHU._children:
    if key.startswith('color_map'):
        del(MapPopPerHU._children[key])
# create population per housing unit by neighborhood map layer
MapPopPerHUH = folium.Choropleth(geo_data=hoodgeoJSON,name="Population per Housing Unit",data=hoodgeo,bins=8,
                                columns=["Neighborhood","Pop per HU (hood)"],key_on="feature.properties.Neighborhood",
                                fill_color="PuBu",fill_opacity=0.7,line_opacity=0.2,nan_fill_opacity=0,show=False,
                                highlight=True,legend_name="Population per Housing Unit (hood)")
folium.features.GeoJsonPopup(fields=['Pop per HU (hood)','Neighborhood']).add_to(MapPopPerHUH.geojson)
folium.features.GeoJson(hoodgeoJSON,name='Labels', # adds hover labels
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['Pop per HU (hood)'],
                                                                aliases = ['Population per Housing Unit'],
                                                                labels=True,sticky=False)).add_to(MapPopPerHUH.geojson)
for key in MapPopPerHUH._children:
    if key.startswith('color_map'):
        del(MapPopPerHUH._children[key])
# create heat map layer
VenueArr = venue_master[['VenLatNum', 'VenLonNum']].values
fg_heat = plugins.HeatMap(VenueArr, radius=15, name="Heat",overlay=True, control=True, show=True, 
                          no_wrap=False)
# create styles for transit map layers
styleT = {'color': 'blue', 'weight': '3', 'opacity': '1'}
highlightT = {'color': 'darkblue', 'weight': '2', 'opacity': '1'}
styleCR = {'color': 'red', 'weight': '3', 'opacity': '1'}
highlightCR = {'color': 'darkred', 'weight': '2', 'opacity': '1'}
styleB = {'color': 'purple', 'weight': '3', 'opacity': '1'}
highlightB = {'color': 'darkpurple', 'weight': '2', 'opacity': '1'}
print("...STYLES AND HIGHLIGHTS CREATED")
# create MBTA T map layer
fg_MBTA_T = folium.FeatureGroup('MBTA T',show=False,z_index_offset=800)
folium.GeoJson(MBTA_RT_Line, name="T Lines", style_function=lambda x: styleT,
               highlight_function=lambda x: highlightT,
               tooltip=folium.features.GeoJsonTooltip(fields=['Line','Route'])).add_to(fg_MBTA_T)
for lat, lng, station, line, route in zip(MBTA_RT_Stop['lat'],MBTA_RT_Stop['lon'],MBTA_RT_Stop['Station'],
                                     MBTA_RT_Stop['Line'], MBTA_RT_Stop['Route']):
    label = '{} ({} Line, {})'.format(station,line,route)
    label = folium.Popup(label, parse_html=True, max_width=150)
    folium.CircleMarker([lat, lng],radius=4,popup=label,color='blue',fill=True,fill_color='blue',fill_opacity=0.5,
                        parse_html=False,name="MBTA T").add_to(fg_MBTA_T)
#create MBTA Commuter Rail map layer
fg_MBTA_CR = folium.FeatureGroup('MBTA Commuter Rail',show=False,z_index_offset=700)
folium.GeoJson(MBTA_Rail_Line, name="CR Lines", style_function=lambda x: styleCR, 
               highlight_function=lambda x: highlightCR,
               tooltip=folium.features.GeoJsonTooltip(fields=['Line'])).add_to(fg_MBTA_CR)
for lat, lng, station, line in zip(MBTA_Rail_Stop['lat'],MBTA_Rail_Stop['lon'],MBTA_Rail_Stop['Station'],
                                     MBTA_Rail_Stop['Line']):
    label = '{} (Commuter Rail {})'.format(station,line)
    label = folium.Popup(label, parse_html=True, max_width=150)
    folium.CircleMarker([lat, lng],radius=4,popup=label,color='red',fill=True,fill_color='red',fill_opacity=0.5,
                        parse_html=False,name="MBTA Commuter Rail").add_to(fg_MBTA_CR)
# create MBTA Bus map layer
fg_MBTA_Bus = folium.FeatureGroup('MBTA Bus Routes',show=False,z_index_offset=600)
folium.GeoJson(MBTA_Bus_Line, name="Bus Lines", style_function=lambda x: styleB, 
               highlight_function=lambda x: highlightB,
               tooltip=folium.features.GeoJsonTooltip(fields=['Route','Headsign'])).add_to(fg_MBTA_Bus)
print("...FEATURES CREATED")

First up is the popular "heat map", which represents density of venues via hotter colors. Use the layer toggle on the upper left of the map to switch the map tile layer to dark monochrome or standard map if that helps you visualize better.

In [ ]:
# heat map
heat_map = folium.Map(location=[latitude, longitude], zoom_start=13, control_scale=True, prefer_canvas=True)
mono_white = folium.raster_layers.TileLayer(tiles='cartodbpositron', name="Light Monochrome", 
                                            overlay=False, control=True, show=True, no_wrap=False,
                                            opacity=1).add_to(heat_map)             # adding light mono tiles
mono_black = folium.raster_layers.TileLayer(tiles='cartodbdark_matter', name="Dark Monochrome", 
                                            overlay=False, control=True, show=False, no_wrap=False,
                                            opacity=1).add_to(heat_map)             # adding dark mono tiles
heat_map.add_child(fg_heat).add_child(MapTract).add_child(MapHoods)                 # adding layers
heat_map.keep_in_front(fg_heat,MapTract,MapHoods)                                   # toggle to keep heat layer in front
plugins.Fullscreen(position="topleft",title="Fullscreen",title_cancel="Exit Fullscreen",
                   force_separate_button=True,).add_to(heat_map)                    # adding fullscreen toggle
folium.LayerControl(position="topleft").add_to(heat_map) 
heat_map

Below is the "fat map", which mushes a whole bunch of layer options into one map. There are layers showing the location of each dining venue in Boston and 1000 foot buffers around each venue, neighborhood and census tract boundaries, T lines and stops, Commuter Rail lines and stops, bus lines, a variety of demographic stats; population, population density, venue count, venue density, population per venue, median household income, and population per housing unit. They get pretty muddled when they're all on, so toggle as you see fit.

In [ ]:
# map with extra herbs and spices
fat_map = folium.Map(location=[latitude, longitude], zoom_start=12, control_scale=True, prefer_canvas=True)
fat_map.add_child(MapHoods).add_child(MapTract).add_child(fg_VenMarkers).add_child(VenBuffer).add_child(MapPop)
fat_map.add_child(MapPopDen).add_child(MapVC).add_child(MapVD).add_child(MapPV).add_child(MapPopInc)
fat_map.add_child(MapPopPerHU).add_child(fg_MBTA_T).add_child(fg_MBTA_CR).add_child(fg_MBTA_Bus)
fat_map.keep_in_front(MapHoods,MapTract,fg_MBTA_Bus,fg_MBTA_CR,fg_MBTA_T,MapPopPerHU,MapPopInc,MapPV,MapPopDen,
                      MapPop,MapVD,MapVC,VenBuffer,fg_VenMarkers)         # periodically this seems to act in reverse
plugins.Fullscreen(position="topleft",title="Fullscreen",title_cancel="Exit Fullscreen",
                   force_separate_button=True,).add_to(fat_map)           # adding fullscreen toggle
folium.LayerControl(position="topleft").add_to(fat_map)                   # adding layer toggle
fat_map

# <span style="color:darkred">Analysis</span> <a name="analysis"></a>

Okay, so we've mapped where everything in Boston is, the people, the venues, the transit lines. We've found our starting point, areas which are under-represented in terms of venue per capita. Now it's time to focus in on identifying areas meeting a defined list of criteria. We'll blanket the city with a grid of hexagons surrounding centroids 1000 feet apart. We'll eliminate any centroids that landed within the gray buffer zones we mapped. We'll eliminate centroids that landed in the water (a possibility given that census tracts do contain water areas). We'll eliminate centroids that landed in unlikely areas for placing a restaurant (school grounds, cemeteries, parks, wetlands). Then we'll eliminate any centroids in tracts with population per venue of less than 700. This will give us locations which are:
<p>1) not near existing venues
<br>2) not in obviously implausible locations
<br>3) in parts of the city under-populated with food venues

### Time For A Hexy Party

In [ ]:
# variables for hex work, X = Lon, Y = Lat
maxX = Bos_maxX32619 # Bos_maxX
minX = Bos_minX32619 # Bos_minX
maxY = Bos_maxY32619 # Bos_maxY
minY = Bos_minY32619 # Bos_minY
centerX = Bos_centerX32619 # Bos_centerX
centerY = Bos_centerY32619 # Bos_centerY
BostonPolygonX = BostonPolygon32619 # BostonPolygon
x_step = 304.8 # .0025 Distance between adjacent hexagons in the row in UM of CRS (304.8m = 1000ft)
print("...HEX VARIABLES SET")

The function below will create our evenly spaced centroids, with the requirement that they must be within the overall polygon of the City of Boston we created up near the top.

In [ ]:
# creating a matrix of points within a geography
def xy_to_lonlat(x, y):
    lonlatTransformer = Transformer.from_crs(32619,4326,always_xy=True) #4326
    lonlat = lonlatTransformer.transform(x, y)
    return lonlat[0], lonlat[1]
k = math.sqrt(3)/2   # Vertical distance coefficient for hexagon grid cells
y_step = x_step * k  # Distance between adjacent hexagons in vertical direction
lat_result = []      # creating empty list
lon_result = []      # creating empty list
xs = []              # creating empty list
ys = []              # creating empty list
num_columns = int((maxX - minX)/x_step) + 1
num_rows = int((maxY - minY)/y_step) + 1
for i in range(0, num_rows):
    y = minY + i * y_step
    x_offset = x_step/2 if i%2==0 else 0
    for j in range(0, num_columns):
        x = minX + j * x_step + x_offset
        lon = x
        lat = y
#        lon, lat = xy_to_lonlat(x, y)
        if boolean_point_in_polygon(Feature(geometry=Point([lon,lat])),Feature(geometry=BostonPolygonX)):
            lat_result.append(lat)
            lon_result.append(lon)
            xs.append(x)
            ys.append(y)
print("...",len(lat_result),'POINTS CREATED')

Next we'll create a function to build the hexagonal polygons around a given set of coordinates.

In [ ]:
# creating hexagon function
def lonlat_to_xy(lon, lat):
    XYTransformer = Transformer.from_crs(4326,32619,always_xy=True) #4326
    xy = XYTransformer.transform(lon, lat)
    return xy[0], xy[1]
def get_hexagon_coordinates(centroid_latlon, side_length_in_meters):
    unit_hexagon_vertices_rotate = np.array([[0, 1], [-math.sqrt(3)/2, 1/2], [-math.sqrt(3)/2, -1/2],
                                       [0, -1], [math.sqrt(3)/2, -1/2], [math.sqrt(3)/2, 1/2]])
#    x, y = lonlat_to_xy(centroid_latlon[1], centroid_latlon[0])        # takes lon/lat and changes to XY
    x, y = (centroid_latlon[1], centroid_latlon[0])                     # skips conversion    
    vertices = unit_hexagon_vertices_rotate
    vertices *= side_length_in_meters
    vertices = [xy_to_lonlat(v[0]+x, v[1]+y) for v in vertices]
    return vertices
print("...FUNCTIONS CREATED")

And next we use the function to create the polygons using the list of centroids we just made.

In [ ]:
# creating hexagons
a = x_step/math.sqrt(3)
hexagons = []
for latlon in zip(lat_result, lon_result):
    vertices = get_hexagon_coordinates(latlon, a) # had been 'a'
    hexagons.append(Polygon(vertices))
print("...",len(hexagons),'HEXAGONS CREATED')

In [ ]:
# working with hex results
# turning lat, lon, x, y results into DF as floats
lat_resultDF = pd.DataFrame([lat_result]).transpose().astype(float)
lat_resultDF.rename(columns={0:"lat (y)",}, inplace=True)
lon_resultDF = pd.DataFrame([lon_result]).transpose().astype(float)
lon_resultDF.rename(columns={0:"lon (x)",}, inplace=True)
xsDF = pd.DataFrame([xs]).transpose().astype(float)
xsDF.rename(columns={0:"x",}, inplace=True)
ysDF = pd.DataFrame([ys]).transpose().astype(float)
ysDF.rename(columns={0:"y",}, inplace=True)
# merging DFs by index
intDF1=lon_resultDF.merge(lat_resultDF, how='outer', left_index=True, right_index=True)
intDF2=intDF1.merge(xsDF, how='outer', left_index=True, right_index=True)
MergePointsDF=intDF2.merge(ysDF, how='outer', left_index=True, right_index=True)
# converting DF to GDF
MergePointsGDF=gpd.GeoDataFrame(MergePointsDF, geometry=gpd.points_from_xy(MergePointsDF.x, MergePointsDF.y),crs=32619)
MergePointsGDF.drop(columns=['x', 'y'], inplace=True)
HexStep1 = MergePointsGDF.shape
MergePointsGDF.head()

In [ ]:
# making GeoJSON from hexes
HexOne_DF = pd.DataFrame([hexagons]).transpose()
HexOne_DF.rename(columns={0:"geometry",}, inplace=True)
# make GeoDataFrame from DataFrame, assign CRS
HexOne_GDF = gpd.GeoDataFrame(HexOne_DF, crs=2249)
# make JSON from GDF
HexOne_GDFJSON = HexOne_GDF.to_json()
HexOne_GDF.shape

Below is our baseline hexmap of Boston, without any of the aforementioned exclusions applied.

In [ ]:
# preliminary hex map
hexone_map = folium.Map(location=[42.31953, -71.08097], zoom_start=12, control_scale=True, prefer_canvas=True)
fg_testhex = folium.GeoJson(HexOne_GDFJSON,show=True,name="Hexagons").add_to(hexone_map)
hexone_map.add_child(MapHoods).add_child(MapTract).add_child(VenBuffer).add_child(fg_VenMarkers)
hexone_map.keep_in_front(MapHoods,MapTract,VenBuffer,fg_testhex,fg_VenMarkers)
plugins.Fullscreen(position="topleft",title="Fullscreen",title_cancel="Exit Fullscreen",
                   force_separate_button=True,).add_to(hexone_map)
folium.LayerControl(position="topleft").add_to(hexone_map) 
hexone_map

### Removing Points in Buffer from Hex List
Now it's time to start carving chunks out of the master hexmap we just made, based on the rules we've laid out. The first step is to take each centroid in the grid and check if it is within the buffer zone multipolygon we made earlier. A new column will be added to the point list to tell us if a given row is in the buffer or not, then we'll remove the offending rows.

In [ ]:
# checking for centroids in the venue buffer zone
InBuffer = []        # creating empty list
for lon, lat in zip(MergePointsGDF['lon (x)'], MergePointsGDF['lat (y)']):
    x = lon #= x
    y = lat #= y
#    lon, lat = xy_to_lonlat(x, y)
    if boolean_point_in_polygon(Feature(geometry=Point([lon,lat])),Feature(geometry=ven_locs_buffer_union32619)):
        InBuffer.append([lat,lon,x,y])
print("...POINTS IN BUFFER IDENTIFIED")

In [ ]:
# taking buffer check results and putting into GDF
InBufferDF = pd.DataFrame(InBuffer)
InBufferDF.rename(columns={0:"lat (y)",1:"lon (x)",2:"x",3:"y"}, inplace=True)
InBufferGDF32619=gpd.GeoDataFrame(InBufferDF, geometry=gpd.points_from_xy(InBufferDF.x, InBufferDF.y),crs=32619)
InBufferGDF32619.drop(columns=['x', 'y'], inplace=True)
InBufferGDF32619['in buffer'] = 'yes'
InBufferGDF32619.reset_index(inplace=True, drop=True)
print("MergePointsGDF:",MergePointsGDF.shape," / InBufferGDF32619:",InBufferGDF32619.shape)

In [ ]:
# working with "in buffer" results
# merging original point list with "in buffer" point list
MergePointsF1GDF32619=MergePointsGDF.merge(InBufferGDF32619, how='outer', on='geometry')
MergePointsF1GDF32619.drop(columns=['lat (y)_y', 'lon (x)_y'], inplace=True)
MergePointsF1GDF32619.rename(columns={"lat (y)_x":"lat (y)","lon (x)_x":"lon (x)"}, inplace=True)
MergePointsF1GDF32619.reset_index(inplace=True, drop=True)
# dropping "in buffer = yes" rows, then dropping "in buffer" column
MergePointsF1GDF32619.drop(MergePointsF1GDF32619.loc[MergePointsF1GDF32619['in buffer']=='yes'].index,inplace=True)
MergePointsF1GDF32619.drop(columns=['in buffer'], inplace=True)
MergePointsF1GDF32619.reset_index(inplace=True, drop=True)
# converting to CRS 4326 for census tract check
MergePointsF1GDF = MergePointsF1GDF32619.to_crs(epsg=4326)
# copying geometry coords to their own columns
MergePointsF1GDF['lon'] = MergePointsF1GDF['geometry'].x
MergePointsF1GDF['lat'] = MergePointsF1GDF['geometry'].y
MergePointsF1GDF=MergePointsF1GDF[['lon','lon (x)','lat','lat (y)','geometry']]
HexStep2 = MergePointsF1GDF.shape
print('MergePointsF1GDF (4326) Type:', type(MergePointsF1GDF), "/ current CRS is:",MergePointsF1GDF.crs)

### Checking Census Tract of Hex List

The Census Bureau uses tract codes in the 9800 range to identify special land-use areas with minimal population, such as parks, wetlands, nature preserves, cemeteries, airports, industrial land, etc. The Boston data bears this out:
<p>9801.01 Harbor Islands
<br>9803 Franklin Park
<br>9807 Stony Brook Reservation
<br>9810 Arnold Arboretum
<br>9811 Forest Hills Cemetery
<br>9812.01 Marine Park and Castle Island
<br>9812.02 Massport Terminal
<br>9813 Logan Airport
<br>9815.01 Charles River Reservation
<br>9815.02 Irving Oil Terminal
<br>9816 Belle Isle Marsh Reservation
<br>9817 Boston Commons
<br>9818 Muddy River and Jamaica Pond

Of these only the Logan Airport tract has a significant restaurant density, however the entire tract is controlled by Massport, and the 80+ venues (including nine Dunkin' shops) are all in the various terminals of the airport. Definitely doesn't meet our "not near existing venues" rule. So we'll remove the hexagon centroids that are in the 98XX tracts.

In [ ]:
# create "getCensusHexInfo" function with Lat+Lng from input "MergePointsF1GDF"
# determines what Census Tract each hex point is located in
# benchmark (4) = Public_AR_Current, vintage (410) = Census2010_Current
def getCensusHexInfo(lons,lonx,lats,laty,geo):
    census_list=[]
    for lng,lnx,lat,lay,geom in zip(lons,lonx,lats,laty,geo):
        # print(ven_id)
        url_batch_geocode = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={}&y={}&benchmark=4&vintage=410&format=json'.format(
            lng,lat)
        census_results = requests.get(url_batch_geocode).json()['result']['geographies']['Census Tracts']
        census_list.append([(lng,lnx,lat,lay,geom,v['POP100'],v['GEOID'],v['INTPTLAT'],v['NAME'],v['INTPTLON'],
                             v['HU100']) for v in census_results])
    census_info = pd.DataFrame([item for loc_list in census_list for item in loc_list])
    census_info.columns = ['lon','lon (x)','lat','lat (y)','geometry','Population','GEOID','INTPTLAT',
                           'NAME','INTPTLON','Housing_Units']
    return(census_info)
print("...'getCensusInfo' FUNCTION CREATED")

In [ ]:
# running getCensusHexInfo
hex_census_raw=getCensusHexInfo(lons=MergePointsF1GDF['lon'],lonx=MergePointsF1GDF['lon (x)'],
                                lats=MergePointsF1GDF['lat'],laty=MergePointsF1GDF['lat (y)'],
                                geo=MergePointsF1GDF['geometry'])
print('...CENSUS TRACT RETRIEVAL COMPLETE')
HexStep3 = hex_census_raw.shape
hex_census_raw.shape

In [ ]:
# hex census cleanup
hex_census=hex_census_raw[['GEOID','NAME','Population','Housing_Units','INTPTLAT','INTPTLON','lon','lon (x)',
                           'lat','lat (y)','geometry']]
hex_census.rename(columns={"GEOID":"GEOID10","NAME":"NAMELSAD10"}, inplace=True) # renaming cols
hex_census.drop(columns=['INTPTLAT','INTPTLON'], inplace=True)                   # dropping unneeded cols
hex_census.drop(hex_census.loc[hex_census['Population']==0].index,inplace=True)  # dropping 0 pop rows
hex_census = hex_census[~hex_census.GEOID10.str.startswith(('2502598'))]         # dropping rows in 98XX tracts
hex_census = hex_census[~hex_census.GEOID10.str.startswith(('250214'))]          # dropping a couple on the border
hex_census.reset_index(inplace=True, drop=True)
HexStep4 = hex_census.shape
hex_census.shape

Using the census tract ID we've pulled, we can tie back to the various bits of demographic etc data we previously assembled, and based on that we'll remove any centroids that are in census tracts below 700 people per venue.

In [ ]:
# merging hex data with tract info via identified census tract
hex_tract_info = pd.merge(hex_census, tractgeo, how='left', on='NAMELSAD10')
# dropping cols
hex_tract_info.drop(columns=['Population_y','Housing_Units','FID','OBJECTID','STATEFP10','State Name','COUNTYFP10',
                             'County Name','TRACTCE10','GEOID10_y','MTFCC10','FUNCSTAT10','ALAND10','AWATER10',
                             'INTPTLAT10','INTPTLON10','Shape_STAr','Shape_STLe','Shape__Area','Shape__Length',
                             'geometry_y','NAME10'], inplace=True)
# renaming cols
hex_tract_info.rename(columns={"GEOID10_x":"GEOID10","Population_x":"Population","geometry_x":"geometry (4326)",
                               "lat":"lat (4326)","lon":"lon (4326)","lat (y)":"y","lon (x)":"x"}, inplace=True)
HexStep5 = hex_tract_info.shape
hex_info=hex_tract_info[['GEOID10','NAMELSAD10','Neighborhood','Population','Population Density','Housing Units',
                         'Pop per HU','Venue Count','Venue Density','Pop per Venue','Median Household Income',
                         'lon (4326)','x','lat (4326)','y','geometry (4326)']]
hex_info.reset_index(inplace=True, drop=True)
HexStep6 = hex_info.shape
hex_info.drop(hex_info.loc[hex_info['Pop per Venue']<700].index,inplace=True)  # dropping low ppv rows
hex_info.reset_index(inplace=True, drop=True)
HexStep7 = hex_info.shape
print("shape changes:",HexStep1,"/",HexStep2,"/",HexStep3,"/",HexStep4,"/",HexStep5,"/",HexStep6,"/",HexStep7)

We started with 1816 centroids, removing those in the buffer zone dropped it to 819, removing those in non-viable tracts dropped it to 460, and limiting to those with Pop per Ven of 700+ dropped it to 253. Let's take a quick look to check if any of the remaining centroids are sketchy, eg. just off shore, in a park or schoolyard, etc. We'll map our remaining centroids and take a quick look at where the centroids landed. For example, that zig-zag of four points in the river just south of the airport... those should probably go.

In [ ]:
# hex map with buffer violations removed
hextemp_map=folium.Map(location=[42.31953, -71.08097],zoom_start=12,control_scale=True,prefer_canvas=True)
fg_hexmarkers = folium.FeatureGroup('Markers',show=True,z_index_offset=500)
for lat, lng, Col1, Col2 in zip(hex_info['lat (4326)'], hex_info['lon (4326)'],hex_info['x'], hex_info['y']):
    label = '{} ({})'.format(Col1,Col2)
    label = folium.Popup(label, parse_html=True, max_width=120)
    folium.CircleMarker([lat, lng],radius=4,popup=label,color='black',fill=True,fill_color='gray',fill_opacity=0.5,
                        parse_html=False,name="Markers").add_to(fg_hexmarkers)
    fg_hexmarkers.add_to(hextemp_map)
hextemp_map.add_child(MapHoods).add_child(MapTract)
plugins.Fullscreen(position="topleft",title="Fullscreen",title_cancel="Exit Fullscreen",
                   force_separate_button=True,).add_to(hextemp_map)
folium.LayerControl(position="topleft").add_to(hextemp_map) 
hextemp_map

As one would expect, the matrix of centroids we generated has some "problem children". The coordinates for those locs have been passed to a separate list and next we'll split the centroids into "ok" and "problem" for one final inspection.

In [ ]:
# "problem hex" work
problem_hex[['b1', 'b2']] = problem_hex['geometry (4326)'].str.strip('POINT ') \
                              .str.strip('()')                                 \
                              .str.split(' ', expand=True)
hex_info[['geotext']]=hex_info['geometry (4326)'].astype(str)
hex_info[['b1', 'b2']] = hex_info['geotext'].str.strip('POINT ') \
                              .str.strip('()')                   \
                              .str.split(' ', expand=True)
hex_info_fix = pd.merge(hex_info, problem_hex, how='left', left_on=['b1','b2'], right_on = ['b1','b2'])
hex_info_fix.drop(columns=['geotext','b1','b2','GEOID10_y','NAMELSAD10_y','Neighborhood_y','Population_y',
                           'Population Density_y','Housing Units_y','Pop per HU_y','Venue Count_y','Venue Density_y',
                           'Pop per Venue_y','Median Household Income_y','lon (4326)_y','x_y','lat (4326)_y','y_y',
                           'geometry (4326)_y'], inplace=True)
hex_info_fix.rename(columns={"GEOID10_x":"GEOID10","NAMELSAD10_x":"NAMELSAD10","Neighborhood_x":"Neighborhood",
                             "Population_x":"Population","Population Density_x":"Population Density",
                             "Housing Units_x":"Housing Units","Pop per HU_x":"Pop per HU",
                             "Venue Count_x":"Venue Count","Venue Density_x":"Venue Density",
                             "Pop per Venue_x":"Pop per Venue","Median Household Income_x":"Median Household Income",
                             "lon (4326)_x":"lon (4326)","x_x":"x","lat (4326)_x":"lat (4326)","y_x":"y",
                             "geometry (4326)_x":"geometry (4326)","geometry_x":"geometry"}, inplace=True)
hex_info_fix.reset_index(inplace=True, drop=True)
problemdots = hex_info_fix[(hex_info_fix.Status == 'problem')]
problemdots.reset_index(inplace=True, drop=True)
okdots = hex_info_fix[(hex_info_fix.Status == 'ok')]
okdots.reset_index(inplace=True, drop=True)
print("problem_hex",problem_hex.shape,"/ hex_info",hex_info.shape,"/ hex_info_fix",hex_info_fix.shape,"/ okdots",
      okdots.shape,"/ problemdots",problemdots.shape)

In [ ]:
# checking for remaining issues, should be no NaN status entries
hex_info_fix.groupby('Status', dropna=False)[["GEOID10"]].count()

Of our 253 remaining centroids, 72 are in places like schoolyards, marshland, etc. Dropping those leaves 181 viable centroids remaining. In the map below, the bad ones are in red, the survivors are in green.

In [ ]:
# creating map of problem hex work
hexfix_map = folium.Map(location=[42.31953, -71.08097], zoom_start=12, control_scale=True, prefer_canvas=True)
fg_prob = folium.FeatureGroup('Problem Dots',show=True,z_index_offset=1000)
for lat, lng, Col1, Col2 in zip(problemdots['lat (4326)'], problemdots['lon (4326)'],problemdots['x'],problemdots['y']):
    label = '{} ({})'.format(Col1,Col2)
    label = folium.Popup(label, parse_html=True, max_width=120)
    folium.CircleMarker([lat, lng],radius=4,popup=label,color='darkred',fill=True,fill_color='red',fill_opacity=0.5,
                        parse_html=False,name="Markers").add_to(fg_prob)
    fg_prob.add_to(hexfix_map)
fg_ok = folium.FeatureGroup('OK Dots',show=True,z_index_offset=1000)
for lat, lng, Col1, Col2 in zip(okdots['lat (4326)'], okdots['lon (4326)'],okdots['x'], okdots['y']):
    label = '{} ({})'.format(Col1,Col2)
    label = folium.Popup(label, parse_html=True, max_width=120)
    folium.CircleMarker([lat, lng],radius=4,popup=label,color='darkgreen',fill=True,fill_color='green',
                        fill_opacity=0.5,parse_html=False,name="Markers").add_to(fg_ok)
    fg_ok.add_to(hexfix_map)
hexfix_map.add_child(MapHoods).add_child(MapTract).add_child(fg_hexmarkers)
hexfix_map.keep_in_front(MapHoods,MapTract,fg_hexmarkers,fg_prob,fg_ok)
plugins.Fullscreen(position="topleft",title="Fullscreen",title_cancel="Exit Fullscreen",
                   force_separate_button=True,).add_to(hexfix_map)
folium.LayerControl(position="topleft").add_to(hexfix_map) 
hexfix_map

Next we'll take our 181 remaining centroids and use K-Means clustering to group them. First we'll chart a series of runs using K=1 thru K=20 to determine how many clusters to make.

In [ ]:
# testing K values for KMeans clustering
locs_for_km = okdots[['lon (4326)','lat (4326)']]

import warnings
warnings.filterwarnings("ignore") # code to suppress unnecessary memory leaks warning

distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 20)
for k in K: # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(locs_for_km)
    kmeanModel.fit(locs_for_km)
    distortions.append(sum(np.min(cdist(locs_for_km, kmeanModel.cluster_centers_,'euclidean'), 
                                  axis=1)) / locs_for_km.shape[0])
    inertias.append(kmeanModel.inertia_)
    mapping1[k] = sum(np.min(cdist(locs_for_km, kmeanModel.cluster_centers_,'euclidean'), 
                             axis=1)) / locs_for_km.shape[0]
    mapping2[k] = kmeanModel.inertia_
plt.figure(figsize=(14,10))
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('Elbow Method for K Determination')
plt.locator_params(axis="x", nbins=20)
plt.show()

It's not the most clear-cut elbow in the world, but at 7 there's a decent flattening, so we'll set our K at 7 and make our clusters.

In [ ]:
# running KMeans based on K=7
kclusters = 7
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(locs_for_km)
kmeans.labels_[0:10] 

In [ ]:
# identifying the centroid of the resulting clusters
cluster_centroids = [(cc[0], cc[1]) for cc in kmeans.cluster_centers_]
cluster_centers = pd.DataFrame(cluster_centroids)
cluster_centers.rename(columns={0:"lon",1:"lat"}, inplace=True)
cluster_centers.reset_index(inplace=True, drop=False)
cluster_centers.rename(columns={"index":"ClusterN"}, inplace=True)
cluster_centers['Cluster']=cluster_centers['ClusterN']+1
cluster_centers.drop(columns=['ClusterN'], inplace=True)
cluster_centers

In [ ]:
# adding cluster numbers to hex centroid list
locs_for_km.insert(0, "Cluster_Labels", kmeans.labels_)
ok_clustered = pd.merge(okdots, locs_for_km, how='outer', left_on=['lon (4326)','lat (4326)'], 
                        right_on = ['lon (4326)','lat (4326)'])
ok_clustered['Cluster']=ok_clustered['Cluster_Labels']+1
cols = ok_clustered.columns.tolist()                    # define a list of column names
cols.insert(0, cols.pop(cols.index('Cluster')))         # move the column name to the beggining
ok_clustered = ok_clustered.reindex(columns= cols)      # then use .reindex() function to reorder
ok_clustered.head()

In [ ]:
# creating final hex centroid geodataframe
HexDotsFinalGDF=gpd.GeoDataFrame(ok_clustered, geometry=gpd.points_from_xy(ok_clustered.x, ok_clustered.y),crs=32619)
print("HexDotsFinalGDF",'Type:',type(HexDotsFinalGDF),"/ current CRS is:",HexDotsFinalGDF.crs,"/ shape:",
      HexDotsFinalGDF.shape)

### Making The Final Hexagons

Okay, we've got our list of centroids filtered down to 181 and assigned to one of seven clusters, now it's time to revist our hexagon function and make our final hexagons...

In [ ]:
# creating final hexagons
a = x_step/math.sqrt(3)
final_hexagons = []
for latlon in zip(HexDotsFinalGDF.y,HexDotsFinalGDF.x):
    vertices = get_hexagon_coordinates(latlon, a)
    final_hexagons.append(Polygon(vertices))
print("...",len(final_hexagons),'FINAL HEXAGONS CREATED')

In [ ]:
# transpose hexes into DataFrame, then GeoDataFrame, then GeoJSON
HexTwo_DF = pd.DataFrame([final_hexagons]).transpose()
HexTwo_DF.rename(columns={0:"geometry",}, inplace=True)
# make GeoDataFrame from DataFrame, assign CRS
HexTwo_GDF = gpd.GeoDataFrame(HexTwo_DF, crs=2249)
# make JSON from GDF
HexTwo_GDFJSON = HexTwo_GDF.to_json()
HexTwo_GDF.shape

Below are our final clusters and hexes. There are a variety of other layers you can toggle, neighborhood and tract boundaries, transit lines, and all the demographic layers.

In [ ]:
# final hex map
hextwo_map = folium.Map(location=[42.31953, -71.08097], zoom_start=12, control_scale=True, prefer_canvas=True)
x = np.arange(kclusters) 
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
fg_centmark = folium.FeatureGroup('Centroids',show=True,z_index_offset=1000)
for lat, lon, cluster in zip(HexDotsFinalGDF['lat (4326)'], HexDotsFinalGDF['lon (4326)'], HexDotsFinalGDF['Cluster']):
    label = folium.Popup('Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,color=rainbow[int(cluster)-1],fill=True,
                        fill_color=rainbow[int(cluster)-1],fill_opacity=0.7,control=True,show=True,
                       name="Centroids").add_to(fg_centmark)   
fg_hex = folium.GeoJson(HexTwo_GDFJSON,name="Hexagons",control=True,show=True)
fg_clusters = folium.FeatureGroup('Cluster Centroids',show=True)
for lat, lon, cluster in zip(cluster_centers['lat'], cluster_centers['lon'], cluster_centers['Cluster']):
    label = folium.Popup('Cluster ' + str(cluster), parse_html=True)
    folium.Circle([lat, lon],radius=1600,popup=label,color='gray',fill=True,
                        fill_color='gray',fill_opacity=0.3,control=True,show=True,
                       name="Cluster Centroids").add_to(fg_clusters)
hextwo_map.add_child(fg_hex).add_child(fg_centmark).add_child(fg_clusters).add_child(fg_VenMarkers)
hextwo_map.add_child(MapHoods).add_child(MapTract).add_child(VenBuffer).add_child(MapPop).add_child(MapPopDen)
hextwo_map.add_child(MapVC).add_child(MapVD).add_child(MapPV).add_child(MapPopInc).add_child(MapPopPerHU)
hextwo_map.add_child(fg_MBTA_T).add_child(fg_MBTA_CR).add_child(fg_MBTA_Bus)
hextwo_map.keep_in_front(MapHoods,MapTract,VenBuffer,MapPopPerHU,MapPopInc,MapPV,MapPopDen,MapPop,MapVD,MapVC,
                         fg_MBTA_Bus,fg_MBTA_CR,fg_MBTA_T,fg_clusters,fg_centmark,fg_hex,fg_VenMarkers)
plugins.Fullscreen(position="topleft",title="Fullscreen",title_cancel="Exit Fullscreen",
                   force_separate_button=True,).add_to(hextwo_map)
folium.LayerControl(position="topleft").add_to(hextwo_map) 
hextwo_map

As expected when we first made the "Population per Venue" bar chart, the neighborhoods in the southwest of the city dominate. There are several areas in the these clusters which are adjacent to T/Commuter Rail stations, which is another plus. Before moving on to results and conclusion, we'll take a quick look at what sort of venues are currently present in our target areas.

### One-Hot Encoding on Categories by Census Tract

For purposes of ranking category types within tracts, I decided to use top three as the number of venue categories to use. In previous one-hot encoding exercises we'd gone as far as using the top 10, however looking at the number of different categories in each tract, I found that going that deep introduced more bad data than it was worth.

The way the standard "most common category" function works, once it runs out of venues and/or distinct categories, it starts filling in empty categories alphabetically. This means that if a tract only has three different categories and you ask for the five most common, number 4 and number 5 aren't real. This became apparent when running the function with "num_top_venues" at 10 and finding that "African" made the top ten for quite a few census tracts even though there are only seven African restaurants in the entire city. Looking at the tracts, 87% of tracts have less than 10 distinct categories, 44% of tracts have less than 5 distinct categories, and 23% of tracts have less than 3 distinct categories. So "top ten" is definitely out.

In [ ]:
# one hot encoding, "freq" is percent of venues in the neighborhood
# 'get_dummies' converts categorical variable into dummy/indicator variables.
vencat_onehot = pd.get_dummies(venue_master[['Broad Category']], prefix="", prefix_sep="")
#vencat_onehot = pd.get_dummies(venue_master[['Venue Category']], prefix="", prefix_sep="")
vencat_onehot.head()
# add neighborhood data back to onehot dataframe
vencat_onehot['NAMELSAD10'] = venue_master['NAMELSAD10']
# move re-inserted column back to first position
cols = vencat_onehot.columns.tolist()                    # define a list of column names
cols.insert(0, cols.pop(cols.index('NAMELSAD10')))       # move the column name to the beggining
vencat_onehot = vencat_onehot.reindex(columns= cols)     # then use .reindex() function to reorder
# grouping
vencat_grouped = vencat_onehot.groupby('NAMELSAD10').mean().reset_index()
vencat_grouped.shape

In [ ]:
# create most common ven function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
print("...'return_most_common_venues' FUNCTION CREATED")

In [ ]:
# find most common venues for each tract
num_top_venues = 3
# adds 'st' to '1st' etc
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['NAMELSAD10']
for ind in np.arange(num_top_venues):
    try: columns.append('{}{} Most Common Cat'.format(ind+1, indicators[ind]))
    except: columns.append('{}th Most Common Cat'.format(ind+1))
# create a new dataframe
NAMELSAD10_venues_sorted = pd.DataFrame(columns=columns)
NAMELSAD10_venues_sorted['NAMELSAD10'] = vencat_grouped['NAMELSAD10']
for ind in np.arange(vencat_grouped.shape[0]):
    NAMELSAD10_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vencat_grouped.iloc[ind, :], num_top_venues)
NAMELSAD10_venues_sorted.shape

The rows below list the three most common venue types, along with some demographic basics, for any census tract which hosts one of our 181 final points. You'll notice a handful of tracts that have no venues at all, which seems like an opportunity.

In [ ]:
# find most common venues for census tracts in the hex zones
OK1H = pd.DataFrame(ok_clustered, copy=True)
OK1H.drop(columns=['lon (4326)','x','lat (4326)','y','geometry (4326)','geometry','Cluster_Labels'], inplace=True)
# the "subset" parameter specifies which columns to match on, otherwise it matches on all fields
OK1H.drop_duplicates(subset=['Cluster','GEOID10','Status'], keep='first', ignore_index=True,inplace=True)
OK1H.sort_values(by=['Cluster','Neighborhood','GEOID10'],ignore_index=True, inplace=True)
OK1H_Tops = pd.merge(OK1H, NAMELSAD10_venues_sorted, how='left', on='NAMELSAD10')
OK1H.drop(columns=['Pop per HU','Status'], inplace=True)
ExistingTypes = OK1H_Tops[['Cluster','Neighborhood','NAMELSAD10','1st Most Common Cat','2nd Most Common Cat',
                           '3rd Most Common Cat','Population','Venue Count','Pop per Venue','Population Density',
                           'Venue Density','Housing Units','Median Household Income']]
ExistingTypes

# <span style="color:darkred">Results and Discussion</span> <a name="results"></a>

The analysis shows that the tourist-heavy northeast of the city around the North End and Downtown is already thick with venues. Where there are tourists, there are always possibilities, but I decided to go a different direction. There are a couple different types of restaurant, and I don't mean cuisine categories; First, there are restaurants you go to as an "occasion", for a birthday or special night out or that sort of thing. For those times when you're willing to spend half an hour or more getting across town just for dinner. Second, there are restaurants you go to because they're convenient or nearby. Doesn't mean they're bad restaurants, it just means sometimes you want to get something to eat without the hassle of travel, tourists, reservations, etc. This is the sort of place I'd suggest for the areas we've identified. A decent local place for the people who live in the area. Maybe just a sandwich shop or bodega, maybe a general "American" restaurant. Pizza places and donut shops already seem well-represented in the target areas, perhaps give the locals a convenient nearby dining option that isn't another pizza shop or Dunkin'.

# <span style="color:darkred">Conclusion</span><a name="conclusion"></a>

The intent of this datawork has been to identify candidate areas for a new restaurant in Boston. Using the data available, we settled on using a metric of population per venue as a guide to locating these candidate areas. We've identified several areas with high population, low venue options, and rapid transit access. A stakeholder would need to then take these candidate areas and do additional investigation (eg. zoning requirements, real estate cost and availability, etc.), however I believe we've created a good jumping-off point for further investigation.